<div class='alert-block alert-info'>
    <br>
    <h1 align="center"><b> Connaissances et raisonnement : rapport de projet </h1>
    <h3 align="center">Résolution d'un RubiksCube par un solveur SAT</h3>
    <h5 align="center">Aymeric CONTI et Noémie GUISNEL</a></h5>
    <br>
</div>

#### SOMMAIRE
 
* **Partie 1 : Rubik's Cube 2×2×2**
  1. Remarques
  2. Notations
  3. Implémentation des couleurs, des directions et des faces du Rubik's Cube
  4. Implémentation des variables du solveur
      - 4.1 Classes abstraites
      - 4.2 Classes concrètes
  5. Implémentation du Rubik's Cube et des fonctions d'affichage 3D
  6. Implémentation des clauses
      - 4.1 Clauses liées aux conditions initiales
      - 4.2 Clauses liées aux transitions
      - 4.3 Clauses liées aux contraintes
      - 4.4 Clauses liées aux conditions d'arrêt 
  7. Résolution du Rubik's Cube

* **Partie 2 : RubiksCube 3×3×3**
  1. Remarques
  2. Ajout des variables liées aux centres et aux arêtes
  3. Implémentation des étapes de résolution
  4. Résolution du Rubik's Cube

* **Partie 3 : RubiksCube 4×4×4**


In [1]:
#!pip install -r requirements.txt

<h1 align="left"> <font color='deepskyblue'> Partie 1 : Rubik's Cube 2×2×2 </font></a></h1>  

Le **Rubik’s Cube 2×2×2** est une version simplifiée du Rubik’s Cube classique 3×3×3 avec uniquement des coins. 

<p align="center">
  <img src="images/RubiksCube2.png" alt="RubiksCube2">
</p>


Pour la suite du rapport, nous identifierons les faces ainsi : 

<p align="center">
  <img src="images/RubiksCube2Faces.png" alt="RubiksCube2Faces">
</p>

Et nous considérerons que les positions (fixes) des cubes du Rubik's Cube sont :

<p align="center">
  <img src="images/RubiksCube2Positions.png" alt="RubiksCube2Positions" width= '300'>
</p>

Le cube du bas, de derrière, à gauche non visible porte le numéro 6.


En revanche, les cubes d'identifiant $id$ eux, sont mobiles et peuvent changer de position. On considère également que l'$id$ des cubes correspond à leur position à l'état final.

Dans cet exemple, tous les cubes ont un $id$ égal à leur position, sauf les cubes d'$id$ 1 et 3, qui ont respectivement une position de 3 et 1 :

<p align="center">
  <img src="images/RubiksCube2IdPos.png" alt="RubiksCube2IdPos" width= '300'>
</p>

### <b> 1. Remarques </b>

Tourner la face droite du cube dans le sens horaire est équivalent à tourner celle de gauche dans le sens anti-horaire. De même pour toutes les faces. 

<p align="center">
  <img src="images/RubiksCube2turned.png" alt="RubiksCube2turned">
</p>


On peut alors restreindre l'ensemble des actions possibles à uniquement 3 faces orthogonales sur les 6 du Rubik's Cube, avec 3 rotations possibles : 
1. Sens horaire (Clockwise)
2. Demi-tour (Halfturn)
3. Sens anti-horaire (Counterclockwise)

Considérons que les 3 faces mobiles sont : Droite, Bas, Derrière. 

<p align="center">
  <img src="images/RubiksCube2Rotations2.png" alt="RubiksCube2Rotations2" width= "300">
</p>

Le cube de devant, en haut à gauche (d'$id = 0$) est alors constamment fixe (ce qui élimine une variable pour notre problème).

### <b>2. Notations</b>

##### <b>Les étapes</b>
On note $t_{max}$ le nombre d'étapes pour résoudre le Rubik's Cube

On note $T = \{0, ..., t_{max}\}$ et $T^* = \{1, ..., t_{max}\}$

##### <b>Les actions</b>
On note $a_{f, d}(t)$ la variable booléenne indiquant si l'action de tourner la face $f \in F = \{Right, Bottom, Back\}$ dans la direction $d \in D = \{Clockwise, Halfturn, Counterclockwise\}$ à l'étape $t \in T^*$ est réalisée. (l'action à l'étape $t = 1$ agit sur l'état $t = 0$ pour arriver dans l'état $t = 1$)

##### <b>Le cube</b>

$C = \{1, ..., 7\}$ est l'ensemble des positions des cubes mobiles du Rubik's Cube. 

$C_{Right} = \{1, 3, 5, 7\}$ est l'ensemble des positions des cubes de la face de droite du Rubik's Cube.

$C_{Bottom} = \{2, 3, 6, 7\}$ est l'ensemble des positions des cubes de la face du bas du Rubik's Cube.

$C_{Back} = \{4, 5, 6, 7\}$ est l'ensemble des positions des cubes de la face de derrière du Rubik's Cube.

$O = \{0, 1, 2\}$ est l'ensemble des orientations possibles des cubes. Considérons que l'orientation $o = 0$ est celle de chaque cube lorsque le Rubik's Cube est fini. 

Exemple des trois orientations possibles pour le cube en haut à droite sur la face de devant :

<p align="center">
  <img src="images/RubiksCube2Orientations.png" alt="RubiksCube2Orientations">
</p>

Le Rubik's Cube est représenté par un ensemble de cubes ayant un $id \in C = \{1, ..., 7\}$ et une orientation $o \in O = \{0, 1, 2\}$. 

Notons $\theta_{c, o}(t)$ la variable booléenne indiquant si le cube à la position $c \in C$ est dans l'orientation $o \in O$ à l'étape $t \in T$ (le cube étant à la bonne orientation ssi $\theta_{c, 0}(t)$ est vrai)

Pour les positions, au lieu de créer $8 \times 8$ variables représentant le fait que la position $c$ possède le cube d'identifiant $id$, on utilise la méthode d'encodage binaire proposé par [ce papier](https://arxiv.org/abs/1105.1436) car chaque position possède un unique $id$ :

Pour cela on utilise $\log_2(8) = 3$ variables pour chaque position (au lieu de 8) en encodant l'id en binaire, par example si le cube à la position 2 est celui d'$id = 6$ alors au lieu d'avoir 8 variables one-hot encoded (01000000) on a simplement (110) soit :

- $x_{c = 2, 0} \wedge x_{c = 2, 1} \wedge \neg x_{c = 2, 2}$

(Cette technique n'a pas été utilisé pour les orientations car cela compliquait les transitions pour seulement passer de 3 variables à 2, elle a néanmoins été utilisé pour les orientations des côtés cf le Rubik's Cube 3x3x3)

### <b>3. Implémentation des couleurs, des directions et des faces du Rubik's Cube</b>

In [ ]:
from enum import Enum
from typing import Literal

CornerPos = Literal[0, 1, 2, 3, 4, 5, 6, 7]
CornerOrientation = Literal[0, 1, 2]

class Color(Enum):
    RED = 0
    ORANGE = 1
    BLUE = 2
    GREEN = 3
    WHITE = 4
    YELLOW = 5

    def to_rgb(self) -> tuple[int, int, int]:
        return {
            Color.RED: (255, 0, 0),
            Color.BLUE: (0, 0, 255),
            Color.GREEN: (0, 255, 0),
            Color.YELLOW: (255, 255, 0),
            Color.ORANGE: (255, 165, 0),
            Color.WHITE: (255, 255, 255),
        }[self]

class Direction(Enum):
    NONE = 0
    CLOCKWISE = 1
    HALF_TURN = 2
    COUNTERCLOCKWISE = 3

    @staticmethod
    def from_str(s: str) -> "Direction":
        return {
            "": Direction.CLOCKWISE,
            "2": Direction.HALF_TURN,
            "'": Direction.COUNTERCLOCKWISE,
        }[s]

    @staticmethod
    def not_none() -> list["Direction"]:
        return [Direction.CLOCKWISE, Direction.HALF_TURN, Direction.COUNTERCLOCKWISE]

    def opposite(self) -> "Direction":
        return {
            Direction.NONE: Direction.NONE,
            Direction.CLOCKWISE: Direction.COUNTERCLOCKWISE,
            Direction.HALF_TURN: Direction.HALF_TURN,
            Direction.COUNTERCLOCKWISE: Direction.CLOCKWISE,
        }[self]

    def to_str(self) -> str:
        return {
            Direction.CLOCKWISE: "",
            Direction.HALF_TURN: "2",
            Direction.COUNTERCLOCKWISE: "'",
        }[self]

    def __lt__(self, other: "Direction") -> bool:
        return self.value < other.value

    def __add__(self, other: "Direction") -> "Direction":
        return Direction((self.value + other.value) % 4)

class Face(Enum):
    LEFT = 0
    RIGHT = 1
    TOP = 2
    BOTTOM = 3
    FRONT = 4
    BACK = 5

    def get_vertices_idx(self) -> list[int]:
        """Vertices are ordered in a clockwise manner (it is used for drawing the cube)."""
        return {
            Face.FRONT: [0, 1, 3, 2],
            Face.BACK: [4, 5, 7, 6],
            Face.LEFT: [0, 4, 6, 2],
            Face.RIGHT: [1, 5, 7, 3],
            Face.TOP: [0, 1, 5, 4],
            Face.BOTTOM: [2, 3, 7, 6],
        }[self]

    @staticmethod
    def from_str(s: str) -> "Face":
        return {
            "F": Face.FRONT,
            "B": Face.BACK,
            "L": Face.LEFT,
            "R": Face.RIGHT,
            "U": Face.TOP,
            "D": Face.BOTTOM,
        }[s]

    def opposite(self) -> "Face":
        return {
            Face.FRONT: Face.BACK,
            Face.BACK: Face.FRONT,
            Face.LEFT: Face.RIGHT,
            Face.RIGHT: Face.LEFT,
            Face.TOP: Face.BOTTOM,
            Face.BOTTOM: Face.TOP,
        }[self]

    def to_str(self) -> str:
        return {
            Face.FRONT: "F",
            Face.BACK: "B",
            Face.LEFT: "L",
            Face.RIGHT: "R",
            Face.TOP: "U",
            Face.BOTTOM: "D",
        }[self]

    def __lt__(self, other: "Face") -> bool:
        return self.value < other.value

### <b> 4. Implémentation des variables du solveur

#### 4.1. Classes abstraites

Nous définissons la classe abstraite `Variable`, qui constitue la base de toutes les variables booléennes modélisant l'état du Rubik's Cube. 

Son rôle est de fournir une structure standardisée pour la gestion des variables dans le solveur, en définissant des méthodes comme `compute_id()`, qui génère un identifiant unique pour chaque variable et utilisable par le solveur, et `n_vars()`, qui calcule le nombre total de variables d'une classe donnée.

In [3]:
from abc import ABC, abstractmethod
from math import ceil, log2
from typing import Iterable, cast
import numpy as np

class Variable(ABC):
    cube_size = 2
    t_max = 11

    def __init__(self, t: int, is_true: bool = True) -> None:
        self.t = t
        self.is_true = is_true
        self.id = self.compute_id()

    def __get_subclass_attrs(self):
        return {
            k: v
            for k, v in self.__dict__.items()
            if k in self.__class__.__init__.__code__.co_varnames
        }

    def copy(self) -> "Variable":
        return self.__class__(**self.__get_subclass_attrs())

    @abstractmethod
    def compute_id(self) -> int: ...
    """Returns the unique integer identifier of the variable"""

    @classmethod
    @abstractmethod
    def n_vars(cls) -> int: ...
    """Returns the number of variables of the class (8*3*(t_max + 1) for the corners positions)"""

    @classmethod
    @abstractmethod
    def from_int(cls, var: int) -> "Variable": ...
    """Returns the variable from its unique integer identifier (inverse of compute_id)"""

    @classmethod
    @abstractmethod
    def offset(cls) -> int: ...
    """Returns the number of variables before the first variable of the class"""
    
    def id_repr(self) -> str:
        """Returns a string representation used for the SAT solver"""
        return ("" if self.is_true else "-") + str(self.id)

    def __neg__(self) -> "Variable":
        neg = self.copy()
        neg.is_true = not neg.is_true
        return neg

    def __invert__(self) -> "Variable":
        return -self

    def __repr__(self) -> str:
        return f"{'' if self.is_true else 'not '}{self.__class__.__qualname__}{self.__get_subclass_attrs()}"

    def __mul__(self, other: int) -> "Variable":
        if other == 1:
            return self
        if other == -1:
            return -self
        raise ValueError(f"Invalid literal {other}")

    def __rmul__(self, other: int) -> "Variable":
        return self * other


Ensuite, `VariableParent` est une autre classe abstraite qui permet d’organiser et de structurer les différentes catégories de cubes présents dans le Rubik’s Cube. Bien que dans un cube 2×2×2, seuls les coins soient présents, l’intérêt de cette classe devient plus évident pour la partie 2 de généralisation au Rubik’s Cube 3×3×3, où il faut gérer aussi les centres et les arêtes du cube. 

Cette classe permet par exemple d'implémenter pour chaque catégorie de cubes 
- la fonction `g` de calcul des coordonnées $c_x$, $c_y$ et $c_z$ à partir de la position $c$
- la fonction `g_inv` de calcul de la position $c$ d'un cube à partir de ses coordonnées
- et la fonction `pos_range` qui génère l'ensemble des positions valides.

In [4]:
class VariableParent(ABC):
    @classmethod
    @abstractmethod
    def n_vars(cls) -> int: ...
    """Returns the number of variables of the class (sum of the number of variables of the children)"""

    @classmethod
    @abstractmethod
    def g(cls, pos: CornerPos) -> tuple[int, int, int]: ...
    """Returns the coordinates (c_x, c_y, c_z) of the corner at position pos"""

    @classmethod
    @abstractmethod
    def g_inv(cls, c_x: int, c_y: int, c_z: int) -> CornerPos: ...
    """Returns the position of the corner at coordinates (c_x, c_y, c_z)"""

    @classmethod
    @abstractmethod
    def n_pos(cls) -> int: ...
    """Returns the number of positions of the class (8 for the corners)"""

    @classmethod
    def pos_range(cls) -> Iterable[CornerPos]:
        """Returns the list of valid positions of the class"""
        return range(cls.n_pos())  # type: ignore

<b> Détail des fonctions de rotations : </b>

<b> - Positions </b>

On note : $r_x : F \times D \times C \rightarrow C$, la fonction qui à chaque position associe la position après la rotation

Soit $c \in C$ et $o \in O$

$c_x, c_y, c_z = g(c)$, les coordonnées de la position $c$

$$
r_x(Right, Clockwise, c) =
\begin{cases}
    g\_inv(c_x, c_z, 1 - c_y) \ \ \text{si} \ \ c \in C_{Right} \\
    c \ \ \text{sinon} \\
\end{cases}
$$
$$
r_x(Bottom, Clockwise, c) =
\begin{cases}
    g\_inv(c_z, c_y, 1 - c_x) \ \ \text{si} \ \ c \in C_{Bottom} \\
    c \ \ \text{sinon} \\
\end{cases}
$$
$$
r_x(Back, Clockwise, c) =
\begin{cases}
    g\_inv(c_y, 1 - c_x, c_z) \ \ \text{si} \ \ c \in C_{Back} \\
    c \ \ \text{sinon} \\
\end{cases}
$$

- $\forall \ f \in F$ :
$$r_x(f, Halfturn, c) = r_x^2(f, Clockwise, c)$$
$$r_x(f, Counterclockwise, c) = r_x^3(f, Clockwise, c)$$

In [5]:
def will_rotate(c_x: int, c_y: int, c_z: int, face: Face, depth: int) -> bool:
    if face == Face.RIGHT:
        return c_x == Variable.cube_size - 1 - depth
    if face == Face.BOTTOM:
        return c_y == Variable.cube_size - 1 - depth
    if face == Face.BACK:
        return c_z == Variable.cube_size - 1 - depth
    raise ValueError(f"Invalid face: {face}")


def rotate_cube(
    c_x: int, c_y: int, c_z: int, face: Face, direction: Direction, depth: int
) -> tuple[int, int, int]:
    assert face in {Face.RIGHT, Face.BOTTOM, Face.BACK}, f"Invalid face: {face}"

    if not will_rotate(c_x, c_y, c_z, face, depth):
        return c_x, c_y, c_z

    def rotate_1(face: Face, c_x: int, c_y: int, c_z: int) -> tuple[int, int, int]:
        if face == Face.RIGHT:
            return c_x, c_z, Variable.cube_size - c_y - 1
        if face == Face.BOTTOM:
            return Variable.cube_size - c_z - 1, c_y, c_x
        if face == Face.BACK:
            return c_y, Variable.cube_size - c_x - 1, c_z
        raise ValueError(f"Invalid face: {face}")

    def rotate_i(
        face: Face, c_x: int, c_y: int, c_z: int, n_rotation: int
    ) -> tuple[int, int, int]:
        assert n_rotation > 0, f"Invalid rotations: {n_rotation}"

        if n_rotation == 1:
            return rotate_1(face, c_x, c_y, c_z)

        return rotate_i(face, *rotate_1(face, c_x, c_y, c_z), n_rotation - 1)

    return rotate_i(face, c_x, c_y, c_z, direction.value)

<b> - Orientations </b>

On note : $r_\theta : F \times D \times C \times O \rightarrow O$, la fonction qui à chaque couple de position orientation associe l'orientation après la rotation

On note $s$ la permutation entre $i$ et $j$
$$s_{i, j} :
\begin{cases}
    O \rightarrow O \\
    o \mapsto
    \begin{cases}
        i & \text{si} \quad o = j \\
        j & \text{si} \quad o = i \\
        o & \text{sinon}
    \end{cases}
\end{cases}$$

Soit $c \in C$ et $o \in O$

- $\forall \ f \in F$
$$r_\theta(f, Halfturn, c, o) = o$$

- $\forall \ d \in \{Clockwise, Counterclockwise\}$
$$
r_\theta(Right, d, c, o) =
\begin{cases}
    s_{0, 2}(o) \ \ \text{si} \ \ c \in C_{Right} \\
    o \ \ \text{sinon} \\
\end{cases}
$$
$$
r_\theta(Bottom, d, c, o) =
\begin{cases}
    s_{0, 1}(o) \ \ \text{si} \ \ c \in C_{Bottom} \\
    o \ \ \text{sinon} \\
\end{cases}
$$
$$
r_\theta(Back, d, c, o) =
\begin{cases}
    s_{1, 2}(o) \ \ \text{si} \ \ c \in C_{Back} \\
    o \ \ \text{sinon} \\
\end{cases}
$$

In [6]:
def rotate_orientation(
    c_x: int, c_y: int, c_z: int, orientation: CornerOrientation, face: Face, direction: Direction, depth: int
) -> tuple[int, int, int, CornerOrientation]:
    if not will_rotate(c_x, c_y, c_z, face, depth):
        return c_x, c_y, c_z, orientation

    new_c_x, new_c_y, new_c_z = rotate_cube(c_x, c_y, c_z, face, direction, depth)

    if direction == Direction.HALF_TURN:
        return new_c_x, new_c_y, new_c_z, orientation

    def s(
        i: CornerOrientation,
        j: CornerOrientation,
        orientation: CornerOrientation,
    ) -> CornerOrientation:
        if orientation == i:
            return j
        if orientation == j:
            return i
        return orientation

    if face == Face.RIGHT:
        new_orientation = s(0, 2, orientation)
    elif face == Face.BOTTOM:
        new_orientation = s(0, 1, orientation)
    elif face == Face.BACK:
        new_orientation = s(1, 2, orientation)
    else:
        raise ValueError(f"Invalid face: {face}")

    return new_c_x, new_c_y, new_c_z, new_orientation

La classe abstraite `VariableState` hérite directement de `Variable` et constitue la base des sous-classes `VariableX` dédiée à la position des cubes et `VariableTheta` dédiée l’orientation des cubes. Elles définissent la méthode `rotate()`, qui applique une rotation et retourne la nouvelle position du cube (pour `VariableX`) ou la nouvelle orientation du cube (pour `VariableTheta`).

In [7]:
class VariableState(Variable, ABC):
    def __init__(self, pos: CornerPos, idx: int, t: int, is_true: bool = True) -> None:
        self.pos: CornerPos = pos
        self.idx: int = idx

        super().__init__(t, is_true)

    @classmethod
    @abstractmethod
    def parent(cls) -> type["VariableParent"]: ...
    """Returns the parent class of the variable (Corners, Edges or Centers)"""

    @classmethod
    @abstractmethod
    def n_idx(cls) -> int: ...
    """Returns the number of indices of the class (3 for the corners positions)"""

    @classmethod
    def pos_to_idx(cls, pos: CornerPos) -> int: # Uniquement utile pour les Rubik's cubes de tailles >= 4 (lorsqu'il existe plusieurs cubes identiques)
        return pos

    @classmethod
    def idx_range(cls) -> Iterable[CornerPos]:
        """Returns the list of valid indices of the class"""
        return range(cls.n_idx())  # type: ignore

    @classmethod
    def n_pos(cls) -> int:
        return cls.parent().n_pos()

    @classmethod
    def pos_range(cls) -> Iterable[CornerPos]:
        return cls.parent().pos_range()

    @classmethod
    def from_int(cls, var: int) -> "VariableState":
        var -= cls.offset()
        n_pos = cls.n_pos()
        n_idx = cls.n_idx()

        return cls(
            var % n_pos,  # type: ignore
            (var // n_pos) % n_idx,  # type: ignore
            (var // (n_idx * n_pos)) % (Variable.t_max + 1),
        )

    @classmethod
    def n_vars(cls) -> int:
        return cls.n_pos() * cls.n_idx() * (Variable.t_max + 1)

    def compute_id(self) -> int:
        return (
            self.offset()
            + self.pos
            + self.idx * self.n_pos()
            + self.t * self.n_idx() * self.n_pos()
        )

    def g(self, pos: CornerPos) -> tuple[int, int, int]:
        return self.parent().g(pos)

    def g_inv(self, c_x: int, c_y: int, c_z: int) -> CornerPos:
        return self.parent().g_inv(c_x, c_y, c_z)

    @abstractmethod
    def rotate(
        self, face: Face, direction: Direction, depth: int
    ) -> "VariableState": ...


class VariableX(VariableState):
    def rotate(self, face: Face, direction: Direction, depth: int) -> "VariableX":
        return self.__class__(
            self.g_inv(*rotate_cube(*self.g(self.pos), face, direction, depth)),
            self.idx,
            self.t + 1,
        )

    @classmethod
    def n_idx(cls) -> int:
        if cls.n_pos() == 0:
            return 0
        return ceil(log2(cls.n_pos()))

    @classmethod
    def encode(cls, decoded_idx: int) -> tuple[int, ...]:
        return tuple(
            1 if s == "1" else -1
            for s in np.binary_repr(decoded_idx, width=cls.n_idx())
        )

    @classmethod
    def from_decoded(
        cls, pos: CornerPos, idx_decoded: int, t: int
    ) -> tuple["VariableState", ...]:
        return tuple(
            sign * cls(pos, cast(CornerPos, idx), t)
            for idx, sign in enumerate(cls.encode(idx_decoded))
        )  # type: ignore


class VariableTheta(VariableState): ...

#### 4.2. Classes concrètes

La classe concrète `Var` regroupe les classes : 

- `Corners` (concrétisation de VariableParent), qui définit comment les coins sont positionnés et orientés à travers les sous-classes `x` (concrétisation de VariableX) et `theta` (concrétisation de VariableTheta). 

- `Actions` implémente Variable pour représenter les rotations du cube, stockant pour chaque transformation la face concernée, la direction, la profondeur et le moment où elle est appliquée. 

In [8]:
class Var:
    faces = [Face.RIGHT, Face.BOTTOM, Face.BACK]
    directions = [Direction.CLOCKWISE, Direction.HALF_TURN, Direction.COUNTERCLOCKWISE]
    depths: list[int] = []

    class Corners(VariableParent):
        class x(VariableX):
            @classmethod
            def offset(cls) -> int:
                return 1

            @classmethod
            def parent(cls) -> type["Var.Corners"]:
                return Var.Corners

        class theta(VariableTheta):
            @classmethod
            def n_idx(cls) -> int:
                return 3

            @classmethod
            def offset(cls) -> int:
                return 1 + Var.Corners.x.n_vars()

            @classmethod
            def parent(cls) -> type["Var.Corners"]:
                return Var.Corners

            def rotate(
                self, face: Face, direction: Direction, depth: int
            ) -> "VariableTheta":
                c_x, c_y, c_z, orientation = rotate_orientation(
                    *self.g(self.pos), cast(CornerOrientation, self.idx), face, direction, depth
                )
                return Var.Corners.theta(self.g_inv(c_x, c_y, c_z), orientation, self.t + 1)

        @classmethod
        def n_vars(cls) -> int:
            return Var.Corners.x.n_vars() + Var.Corners.theta.n_vars()

        @classmethod
        def g(cls, pos: CornerPos) -> tuple[int, int, int]:
            return (
                (pos % 2) * (Variable.cube_size - 1),
                ((pos // 2) % 2) * (Variable.cube_size - 1),
                ((pos // 4) % 2) * (Variable.cube_size - 1),
            )

        @classmethod
        def g_inv(cls, c_x: int, c_y: int, c_z: int) -> CornerPos:
            return cast(
                CornerPos,
                int(c_x != 0) + 2 * int(c_y != 0) + 4 * int(c_z != 0),
            )

        @classmethod
        def n_pos(cls) -> int:
            return 8
        
        @classmethod
        def pos_range(cls) -> Iterable[CornerPos]:
            return range(
                1, Var.Corners.n_pos()
            )  # Exclude origin (because it does not move) # type: ignore


    class Actions(Variable):
        def __init__(
            self,
            face: Face,
            direction: Direction,
            depth: int,
            t: int,
            is_true: bool = True,
        ):
            self.face = face
            self.direction = direction
            self.depth = depth

            super().__init__(t, is_true)

        def compute_id(self) -> int:
            return (
                Var.Corners.n_vars()
                + Var.directions.index(self.direction)
                + Var.faces.index(self.face) * 3
                + self.depth * 9
                + self.t * 9 * (Variable.cube_size - 1)
                + 1
            )

        @classmethod
        def n_vars(cls) -> int:
            return 9 * (Variable.t_max + 1) * (Variable.cube_size - 1)
        
        @classmethod
        def offset(cls) -> int:
            return 1 + Var.Corners.n_vars()

        @classmethod
        def from_int(cls, var: int) -> "Var.Actions":
            var -= cls.offset()

            return Var.Actions(
                Var.faces[(var // 3) % 3],
                Var.directions[var % 3],
                (var // 9) % (Variable.cube_size - 1),
                (var // (9 * (Variable.cube_size - 1))),
            )

        def get_params(self) -> tuple[Face, Direction, int, int]:
            return self.face, self.direction, self.depth, self.t

    @staticmethod
    def n_vars() -> int:
        return (
            Var.Corners.n_vars()
            + Var.Actions.n_vars()
        )

    @staticmethod
    def from_int(var: int) -> Variable:
        var_class: list[type[Variable]] = [
            Var.Corners.x,
            Var.Corners.theta,
            Var.Actions,
        ]

        lower_bound = 1
        for cls in var_class:
            n_vars = cls.n_vars()

            if lower_bound <= var < lower_bound + n_vars:
                return cls.from_int(var)
            lower_bound += n_vars

        raise ValueError(f"Invalid variable: {var}")


### <b> 5. Implémentation du Rubik's Cube et des fonctions d'affichage 3D </b>

Le Rubik's Cube est représenté par 6 matrices de taille $n \times n$ représentant une face chacune.

Les détails de cette fonction sont assez peu utiles à la compréhension de notre implémentation pour la résolution du Rubik's Cube.

In [ ]:
import pygame
import random
import re
import imageio

from itertools import pairwise
from math import cos, sin, radians

WIDTH, HEIGHT = 600, 600
X, Y, Z = 0, 1, 2

class RubiksCube:
    """
    ### Fonctions :
    - rotate(face: Face, direction: Direction)
    - shuffle(n: int = 100)
    - apply_rotations(self, rotations: list[str]): Apply a list of rotations using standard notation
        - example : ["U'", "F2", "B", "L'", "R2"]
        - Up (anticlockwise) / Front (2x) / Back (clockwise) / Left (anticlockwise) / Right (2x)
    - show()
        - run a simulator to view the rubik's cube in 3D
    - animate(rotations: list[str], speed: float = 1)
        - run a simulator to view the rubik's cube in 3D with animations
    """

    def __init__(self, size: int) -> None:
        self.size = size
        self.faces = {
            Face.FRONT: np.full((size, size), Color.WHITE.value, dtype=np.int8),
            Face.BACK: np.full((size, size), Color.YELLOW.value, dtype=np.int8),
            Face.LEFT: np.full((size, size), Color.BLUE.value, dtype=np.int8),
            Face.RIGHT: np.full((size, size), Color.GREEN.value, dtype=np.int8),
            Face.TOP: np.full((size, size), Color.RED.value, dtype=np.int8),
            Face.BOTTOM: np.full((size, size), Color.ORANGE.value, dtype=np.int8),
        }

    def __up_face_and_slice(self, face: Face, depth: int) -> tuple[Face, slice]:
        return {
            Face.FRONT: (Face.TOP, np.s_[:, self.size - 1 - depth]),
            Face.BACK: (Face.TOP, np.s_[::-1, depth]),
            Face.LEFT: (Face.TOP, np.s_[depth, :]),
            Face.RIGHT: (Face.TOP, np.s_[self.size - 1 - depth, ::-1]),
            Face.TOP: (Face.BACK, np.s_[::-1, depth]),
            Face.BOTTOM: (Face.FRONT, np.s_[:, self.size - 1 - depth]),
        }[face]

    def __bottom_face_and_slice(self, face: Face, depth: int) -> tuple[Face, slice]:
        return {
            Face.FRONT: (Face.BOTTOM, np.s_[::-1, depth]),
            Face.BACK: (Face.BOTTOM, np.s_[:, self.size - 1 - depth]),
            Face.LEFT: (Face.BOTTOM, np.s_[depth, :]),
            Face.RIGHT: (Face.BOTTOM, np.s_[self.size - 1 - depth, ::-1]),
            Face.TOP: (Face.FRONT, np.s_[::-1, depth]),
            Face.BOTTOM: (Face.BACK, np.s_[:, self.size - 1 - depth]),
        }[face]

    def __left_face_and_slice(self, face: Face, depth: int) -> tuple[Face, slice]:
        return {
            Face.FRONT: (Face.LEFT, np.s_[self.size - 1 - depth, ::-1]),
            Face.BACK: (Face.RIGHT, np.s_[self.size - 1 - depth, ::-1]),
            Face.LEFT: (Face.BACK, np.s_[self.size - 1 - depth, ::-1]),
            Face.RIGHT: (Face.FRONT, np.s_[self.size - 1 - depth, ::-1]),
            Face.TOP: (Face.LEFT, np.s_[::-1, depth]),
            Face.BOTTOM: (Face.LEFT, np.s_[:, self.size - 1 - depth]),
        }[face]

    def __right_face_and_slice(self, face: Face, depth: int) -> tuple[Face, slice]:
        return {
            Face.FRONT: (Face.RIGHT, np.s_[depth, :]),
            Face.BACK: (Face.LEFT, np.s_[depth, :]),
            Face.LEFT: (Face.FRONT, np.s_[depth, :]),
            Face.RIGHT: (Face.BACK, np.s_[depth, :]),
            Face.TOP: (Face.RIGHT, np.s_[::-1, depth]),
            Face.BOTTOM: (Face.RIGHT, np.s_[:, self.size - 1 - depth]),
        }[face]

    def __rotate_clockwise(self, face: Face, depth: int) -> None:
        if depth == 0:
            self.faces[face] = np.rot90(self.faces[face], k=1)  # type: ignore

        up_face, up_slice = self.__up_face_and_slice(face, depth)
        bottom_face, bottom_slice = self.__bottom_face_and_slice(face, depth)
        left_face, left_slice = self.__left_face_and_slice(face, depth)
        right_face, right_slice = self.__right_face_and_slice(face, depth)

        up_color = self.faces[up_face][up_slice].copy()
        self.faces[up_face][up_slice] = self.faces[left_face][left_slice]
        self.faces[left_face][left_slice] = self.faces[bottom_face][bottom_slice]
        self.faces[bottom_face][bottom_slice] = self.faces[right_face][right_slice]
        self.faces[right_face][right_slice] = up_color

    def can_rotate(self, face: Face, depth: int) -> bool:
        if depth < 0:
            return False

        if face in (Face.FRONT, Face.BACK) and depth >= self.size - 1:
            return False
        if face in (Face.LEFT, Face.RIGHT) and depth >= self.size - 1:
            return False
        if face in (Face.TOP, Face.BOTTOM) and depth >= self.size - 1:
            return False

        return True

    def rotate(self, face: Face, direction: Direction, depth: int) -> None:
        assert self.can_rotate(face, depth), "Cannot rotate face"

        for _ in range(direction.value):
            self.__rotate_clockwise(face, depth)

    def shuffle(self, n: int = 100, replace_origin: bool = False) -> list[str]:
        faces = list(Face)
        directions = Direction.not_none()
        depths = list(range(self.size))

        moves_str: list[str] = []
        last_face: Face | None = None
        last_depth: int | None = None

        for _ in range(n):
            face = random.choice(faces)
            direction = random.choice(directions)
            depth = random.choice(depths)

            while not self.can_rotate(face, depth) or (
                face == last_face and depth == last_depth
            ):
                face = random.choice(faces)
                direction = random.choice(directions)
                depth = random.choice(depths)

            last_face = face
            last_depth = depth
            self.rotate(face, direction, depth)

            moves_str.append(RubiksCube.move_to_str(face, direction, depth))

        if replace_origin:
            self.replace_origin()

        return moves_str

    def rotate_whole_cube(self, face: Face, direction: Direction) -> None:
        self.rotate(face.opposite(), direction.opposite(), 0)
        for i in range(self.size - 1):
            self.rotate(face, direction, i)

    def __origin(self) -> tuple[CornerPos, CornerOrientation]:
        for pos in Var.Corners.pos_range():
            idx, o = get_vars_from_corner_pos(self, pos)  # This function will be defined later # type: ignore
            if idx == 0:
                return pos, o
        return get_vars_from_corner_pos(self, 0)  # type: ignore

    def replace_origin(self) -> None:
        pos, o = self.__origin()
        if pos == 0 and o == 0:
            return

        x, y, z = Var.Corners.g(pos)
        
        x_placed = x == 0
        y_placed = y == 0
        z_placed = z == 0

        if o == 0 and not z_placed:
            self.rotate_whole_cube(Face.BOTTOM, Direction.HALF_TURN)
        elif o == 1:
            self.rotate_whole_cube(
                Face.BOTTOM,
                Direction.CLOCKWISE if x_placed else Direction.COUNTERCLOCKWISE,
            )
        elif o == 2:
            self.rotate_whole_cube(
                Face.RIGHT,
                Direction.COUNTERCLOCKWISE if y_placed else Direction.CLOCKWISE,
            )

        pos, o = self.__origin()
        assert o == 0, f"Origin not oriented correctly: {o}"

        x, y, z = Var.Corners.g(pos)
        x_placed = x == 0
        y_placed = y == 0
        z_placed = z == 0

        assert z_placed, f"Origin not placed correctly: {pos}"

        match (x_placed, y_placed):
            case (False, False):
                self.rotate_whole_cube(Face.FRONT, Direction.HALF_TURN)
            case (True, False):
                self.rotate_whole_cube(Face.FRONT, Direction.CLOCKWISE)
            case (False, True):
                self.rotate_whole_cube(Face.FRONT, Direction.COUNTERCLOCKWISE)

        pos, o = self.__origin()
        assert pos == 0, f"Origin not placed correctly: {pos}"

    @staticmethod
    def reverse_moves(moves: list[str]) -> list[str]:
        reverse_moves = []
        for face, direction, depth in RubiksCube.parse_moves(moves)[::-1]:
            reverse_moves.append(
                RubiksCube.move_to_str(face, direction.opposite(), depth)
            )
        return reverse_moves

    @staticmethod
    def move_to_str(face: Face, direction: Direction, depth: int) -> str:
        return f"{face.to_str()}{direction.to_str()}" + "{" + str(depth) + "}"

    @staticmethod
    def parse_move(move: str) -> tuple[Face, Direction, int]:
        face = Face.from_str(move[0])
        move = move[1:]
        depth = re.search(r"{(\d+)}", move)

        if depth:
            depth = int(depth.group(1))
            move = move.replace("{" + str(depth) + "}", "")
        else:
            depth = 0

        return face, Direction.from_str(move), depth

    @staticmethod
    def parse_moves(moves: list[str]) -> list[tuple[Face, Direction, int]]:
        return [RubiksCube.parse_move(move) for move in moves]

    def apply_rotations(self, rotations: list[str]) -> None:
        for rotation in rotations:
            self.rotate(*self.parse_move(rotation))

    def __draw_face(
        self,
        screen: pygame.surface.Surface,
        coords: list[tuple[float, float, float]],
        color: Color,
    ) -> None:
        def project_3d_to_2d(point: tuple[float, float, float], scale: float = 200):
            """Convert 3D point to 2D screen coordinates with perspective projection"""
            x, y, z = point
            factor = scale / (z + 4)  # Simple perspective division
            screen_x = int(WIDTH / 2 + x * factor)
            screen_y = int(HEIGHT / 2 + y * factor)
            return screen_x, screen_y

        screen_positions = [project_3d_to_2d(coord) for coord in coords]
        pygame.draw.polygon(screen, color.to_rgb(), screen_positions)
        pygame.draw.polygon(screen, (0, 0, 0), screen_positions, 2)

    def __draw(
        self,
        screen: pygame.surface.Surface,
        angle_x: float,
        angle_y: float,
        rotating_face: Face | None = None,
        rotating_depth: int = 0,
        rotating_angle: float = 0,
    ) -> None:
        """Draw the 3D Rubik's Cube"""

        def rotate_x(
            point: tuple[float, float, float], angle: float
        ) -> tuple[float, float, float]:
            """Rotate a point in 3D space around X-axis"""
            x, y, z = point
            angle = radians(angle)
            y, z = y * cos(angle) - z * sin(angle), y * sin(angle) + z * cos(angle)
            return x, y, z

        def rotate_y(
            point: tuple[float, float, float], angle: float
        ) -> tuple[float, float, float]:
            """Rotate a point in 3D space around Y-axis"""
            x, y, z = point
            angle = radians(angle)
            x, z = x * cos(angle) + z * sin(angle), -x * sin(angle) + z * cos(angle)
            return x, y, z

        def rotate_z(
            point: tuple[float, float, float], angle: float
        ) -> tuple[float, float, float]:
            """Rotate a point in 3D space around Z-axis"""
            x, y, z = point
            angle = radians(angle)
            x, y = x * cos(angle) - y * sin(angle), x * sin(angle) + y * cos(angle)
            return x, y, z

        def rotate_points_on_face(
            points: list[tuple[float, float, float]],
            face: Face | None,
            depth: int,
            angle: float,
            x: int,
            y: int,
            z: int,
        ) -> list[tuple[float, float, float]]:
            if face is None:
                return points

            if face == Face.LEFT and x == depth:
                return [rotate_x(point, angle) for point in points]
            if face == Face.RIGHT and x == self.size - 1 - depth:
                return [rotate_x(point, angle) for point in points]
            if face == Face.TOP and y == depth:
                return [rotate_y(point, angle) for point in points]
            if face == Face.BOTTOM and y == self.size - 1 - depth:
                return [rotate_y(point, angle) for point in points]
            if face == Face.FRONT and z == depth:
                return [rotate_z(point, angle) for point in points]
            if face == Face.BACK and z == self.size - 1 - depth:
                return [rotate_z(point, angle) for point in points]

            return points

        faces: list[tuple[list[tuple[float, float, float]], int]] = []

        for cube_x, (x0, x1) in enumerate(
            pairwise(np.linspace(-1, 1, self.size + 1))
        ):
            for cube_y, (y0, y1) in enumerate(
                pairwise(np.linspace(-1, 1, self.size + 1))
            ):
                for cube_z, (z0, z1) in enumerate(
                    pairwise(np.linspace(-1, 1, self.size + 1))
                ):
                    points = [
                        (x0, y0, z0),
                        (x1, y0, z0),
                        (x0, y1, z0),
                        (x1, y1, z0),
                        (x0, y0, z1),
                        (x1, y0, z1),
                        (x0, y1, z1),
                        (x1, y1, z1),
                    ]

                    points = rotate_points_on_face(
                        points,
                        rotating_face,
                        rotating_depth,
                        rotating_angle,
                        cube_x,
                        cube_y,
                        cube_z,
                    )
                    points = [rotate_x(point, angle_x) for point in points]
                    points = [rotate_y(point, angle_y) for point in points]

                    if cube_x == 0:
                        faces.append(
                            (
                                [points[idx] for idx in Face.LEFT.get_vertices_idx()],
                                self.faces[Face.LEFT][-cube_z - 1, cube_y],
                            )
                        )
                    if cube_x == self.size - 1:
                        faces.append(
                            (
                                [points[idx] for idx in Face.RIGHT.get_vertices_idx()],
                                self.faces[Face.RIGHT][cube_z, cube_y],
                            )
                        )
                    if cube_y == 0:
                        faces.append(
                            (
                                [points[idx] for idx in Face.TOP.get_vertices_idx()],
                                self.faces[Face.TOP][cube_x, -cube_z - 1],
                            )
                        )
                    if cube_y == self.size - 1:
                        faces.append(
                            (
                                [points[idx] for idx in Face.BOTTOM.get_vertices_idx()],
                                self.faces[Face.BOTTOM][cube_x, cube_z],
                            )
                        )
                    if cube_z == 0:
                        faces.append(
                            (
                                [points[idx] for idx in Face.FRONT.get_vertices_idx()],
                                self.faces[Face.FRONT][cube_x, cube_y],
                            )
                        )
                    if cube_z == self.size - 1:
                        faces.append(
                            (
                                [points[idx] for idx in Face.BACK.get_vertices_idx()],
                                self.faces[Face.BACK][-cube_x - 1, cube_y],
                            )
                        )

        faces_ordered = sorted(
            faces,
            key=lambda face: sum(coord[Z] for coord in face[0]),
            reverse=True,
        )

        for coords, color in faces_ordered:
            self.__draw_face(screen, coords, Color(color))

    def show(
        self,
        screen_size: tuple[int, int] = (600, 600),
        background_color: tuple[int, int, int] = (30, 30, 30),
    ) -> None:
        self.animate([], screen_size, background_color, speed=0)

    def animate(
        self,
        rotations: list[tuple[Face, Direction, int]],
        screen_size: tuple[int, int] = (600, 600),
        background_color: tuple[int, int, int] = (30, 30, 30),
        speed: float = 1,
        fps: int = 30,
        recording: bool = False,
    ) -> None:
        pygame.init()
        screen = pygame.display.set_mode(screen_size)
        pygame.display.set_caption("3D Rubik's Cube")
        clock = pygame.time.Clock()

        running = True
        angle_x, angle_y = 30, 30
        rotating = False
        last_mouse_pos = None

        rotating_face = None
        rotating_depth = 0
        rotating_angle = 0
        rotation_idx = 0
        angle_direction = 1
        target_angle = 0

        frames = []
        record_finished = False

        while running:
            screen.fill(background_color)

            if rotation_idx < len(rotations):
                face, direction, depth = rotations[rotation_idx]

                if rotating_face is None:
                    rotating_face = face
                    rotating_depth = depth
                    rotating_angle = 0

                    angle_direction = {
                        Direction.CLOCKWISE: 1,
                        Direction.HALF_TURN: 2,
                        Direction.COUNTERCLOCKWISE: -1,
                    }[direction] * {
                        Face.FRONT: 1,
                        Face.BACK: -1,
                        Face.LEFT: 1,
                        Face.RIGHT: -1,
                        Face.TOP: 1,
                        Face.BOTTOM: -1,
                    }[face]

                    target_angle = 90 * angle_direction

            else:
                record_finished = True

            if rotating_face is not None:
                rotating_angle += speed * angle_direction

                if abs(rotating_angle) >= abs(target_angle):
                    self.rotate(face, direction, rotating_depth)  # type: ignore
                    rotating_face = None
                    rotation_idx += 1

            self.__draw(
                screen, angle_x, angle_y, rotating_face, rotating_depth, rotating_angle
            )

            if recording and not record_finished:
                frame = pygame.surfarray.array3d(pygame.display.get_surface())
                frame = np.transpose(frame, (1, 0, 2))
                frames.append(frame)

            pygame.display.flip()
            clock.tick(fps)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    rotating = True
                    last_mouse_pos = pygame.mouse.get_pos()
                elif event.type == pygame.MOUSEBUTTONUP:
                    rotating = False
                    last_mouse_pos = None

            if rotating:
                current_mouse_pos = pygame.mouse.get_pos()
                if last_mouse_pos:
                    dx = last_mouse_pos[0] - current_mouse_pos[0]
                    dy = last_mouse_pos[1] - current_mouse_pos[1]

                    angle_x -= dy * 0.5
                    angle_y += dx * 0.5

                    last_mouse_pos = current_mouse_pos

        if recording and frames:
            imageio.mimsave("output4.gif", frames, fps=fps)

        pygame.quit()
    def copy(self) -> "RubiksCube":
        new_cube = RubiksCube(self.size)
        new_cube.faces = {face: self.faces[face].copy() for face in self.faces}
        return new_cube

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, RubiksCube):
            return False

        return all(
            np.array_equal(self.faces[face], other.faces[face]) for face in self.faces
        )

pygame 2.6.1 (SDL 2.28.4, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


### <b> 6. Implémentation des clauses </b>

L'objectif est maintenant de formuler les clauses logiques à partir des variables binaires définies plus haut, nécessaires pour le solveur SAT.

Petite remarque : L'utilisation de nom pour les clauses était uniquement utile pour la phase de debug.

In [10]:
Clause = list[Variable]
NamedClause = tuple[str, Clause]

#### <b> 6.1 Clauses liées aux conditions initiales </b> 

<b>  Positions initiales : </b>

Chaque position $c$ possède une cube spécifique d'identifiant $id_0$ dans l’état mélangé du cube à $t = 0$ :

En notant $id_0 = id_0^0 id_0^1 id_0^2$ sa représentation binaire et $\odot$ l'opérateur `xnor`, on a :
$$\forall c \in C \quad \bigwedge_{i \in \{0, 1, 2\}} id_0^i \odot x_{c, i}(t=0)$$

Par example si à la postion initiale le cube en position $c = 3$ est celui d'$id = 1 \ \text{(001 en binaire)}$ on a :
$$\neg x_{c, 0}(t=0) \wedge \neg x_{c, 1}(t=0) \wedge x_{c, 2}(t=0)$$

<b> Orientations initiales : </b>

De même, chaque position possède une orientation spécifique $\theta_{0}$ à $t = 0$. On impose donc :

$$\forall c \in C, \quad \theta_{c, \theta_0}(t=0) \quad \text{et} \quad \neg \theta_{c, \theta \neq \theta_0}$$

Ces clauses assurent que le Rubik’s Cube est bien modélisé selon son état de départ avant que la résolution ne commence.

La fonction $\text{get\_vars\_from\_corner\_pos}()$ nous permet d'obtenir $id_0$ et $\theta_{0}$ pour un position $c$

Remarque pour le calcul de $\theta_{0}$ des cubes du Rubik's Cube mélangé :

On regarde sur quel axe est la face blanche ou jaune (il y en a une et une seule par coin).
Si elle est sur l’axe z (blanc / jaune) $\rightarrow$ o = 0
Si elle est sur l’axe x (bleu / vert) $\rightarrow$ o = 1
Si elle est sur l’axe y (rouge / orange) $\rightarrow$ o = 2

Exemples : 

<p align="center">
  <img src="images/RubiksCube2Orientations2.png" alt="RubiksCube2Orientations2" width="700">
</p>



In [11]:
def get_vars_from_corner_pos(
        cube: RubiksCube, pos: CornerPos
    ) -> tuple[CornerPos, CornerOrientation]:
        x, y, z = Var.Corners.g(pos)

        colors = [
            Color(
                cube.faces[Face.FRONT if z == 0 else Face.BACK][
                    x if z == 0 else (-x - 1), y
                ]
            ),
            Color(
                cube.faces[Face.LEFT if x == 0 else Face.RIGHT][
                    (-z - 1) if x == 0 else z, y
                ]
            ),
            Color(
                cube.faces[Face.TOP if y == 0 else Face.BOTTOM][
                    x, (-z - 1) if y == 0 else z
                ]
            ),
        ]

        idx = Var.Corners.g_inv(
            int(Color.GREEN in colors) * (cube.size - 1),
            int(Color.ORANGE in colors) * (cube.size - 1),
            int(Color.YELLOW in colors) * (cube.size - 1),
        )
        orientation = cast(
            CornerOrientation,
            np.argmax([color in (Color.WHITE, Color.YELLOW) for color in colors]),
        )

        return idx, orientation

In [12]:
def generate_initial_clauses(cube: RubiksCube) -> list[NamedClause]:
    clauses: list[NamedClause] = []

    for pos in Var.Corners.pos_range():
        idx, theta = get_vars_from_corner_pos(cube, pos)

        for var in Var.Corners.x.from_decoded(pos, idx, 0):
            clauses.append(("Initial state position", [var]))

        for orientation in Var.Corners.theta.idx_range():
            clauses.append(
                (
                    "Initial state orientation",
                    [Var.Corners.theta(pos, orientation, 0, orientation == theta)],
                )
            )

    return clauses

#### <b> 6.2 Clauses liées aux transitions </b>

Les transitions changent à la fois l'orientation et la postion des cubes qu'elles affectent.

En notant :

- $c' = r_x(f, d, c)$, la nouvelle position après la rotation

- $o' = r_\theta(f, d, c, o)$, la nouvelle orientation après la rotation

On a $\forall \ t \in T^*$ :

<b> Pour les positions : </b>
- $\forall \ f \in F, \ \forall \ d \in D, \ \forall \ c \in C, \ \forall \ i \in \{0, 1, 2\}$:
$$a_{f, d}(t) \Rightarrow \Big( x_{c', i}(t) = x_{c, i}(t - 1) \Big)$$

Soit :
$$
\begin{cases}
    x_{c', i}(t) \ \vee \ \neg x_{c, i}(t - 1) \ \vee \ \neg a_{f, d}(t) \\
    \neg x_{c', i}(t) \ \vee \ x_{c, i}(t - 1) \ \vee \ \neg a_{f, d}(t) \\
\end{cases}
$$

<b> Pour les orientations : </b>
- $\forall \ f \in F, \ \forall \ d \in D, \ \forall \ c \in C, \ \forall \ o \in O$ :
$$
a_{f, d}(t) \Rightarrow \Big( \theta_{c', o'}(t) = \theta_{c, o}(t - 1) \Big) \\
$$

Soit :
$$
\begin{cases}
    \theta_{c', o'}(t) \ \vee \ \neg \theta_{c, o}(t - 1) \ \vee \ \neg a_{f, d}(t) \\
    \neg \theta_{c', o'}(t) \ \vee \ \theta_{c, o}(t - 1) \ \vee \ \neg a_{f, d}(t) \\
\end{cases}
$$

In [13]:
from itertools import product

def generate_transitions_clauses_for(
    state: type[VariableState], action: Var.Actions
) -> list[NamedClause]:
    clauses: list[NamedClause] = []

    for c in state.pos_range():
        for idx in state.idx_range():
            var = state(c, idx, action.t - 1)
            var_prime = var.rotate(action.face, action.direction, action.depth)

            clauses.append(
                (
                    f"Transition {state.__name__}, case_cube {c}, {action.face}, {action.direction}, depth {action.depth}, temps {action.t}, clause 1",
                    [var_prime, -var, -action],
                )
            )
            clauses.append(
                (
                    f"Transition {state.__name__}, case_cube {c}, {action.face}, {action.direction}, depth {action.depth}, temps {action.t}, clause 2",
                    [-var_prime, var, -action],
                )
            )

    return clauses

def generate_all_transitions_clauses(actions: set[tuple[Face, Direction, int]]) -> list[NamedClause]:
    """
    Génère les clauses pour les transitions.
    """
    clauses: list[NamedClause] = []

    for t in range(1, Variable.t_max + 1):
        for f, d, depth in actions:
            action = Var.Actions(f, d, depth, t)
            
            clauses += generate_transitions_clauses_for(Var.Corners.x, action)
            clauses += generate_transitions_clauses_for(Var.Corners.theta, action)

    return clauses

#### <b> 6.3 Clauses liées aux contraintes </b>

On ne peut évidemment pas faire plusieurs rotations au même instant $t$ d'où :
$$
\forall \ t \in T^* : \bigwedge_{
    \substack{
        (f, d), (f', d') \in F \times D\\
        f, d \ < \ f', d'\\
    }
} \neg a_{f, d}(t) \vee \neg a_{f', d'}(t)$$

Pour simplifier les transitions on force également le solveur à prendre une action à chaque étape :
$$
\forall \ t \in T^* : \bigvee_{(f, d) \in F \times D} a_{f, d}(t)
$$

En effet, garantir une action permet de garantir que chaque position et orientation sera unique car chaque transition définit tout les états des variables pour l'état $t + 1$

L'action d'attendre pouvant être effectué en faisant un coup inutile avant le dernier coup (par example si on veut tourner dans le sens horaire on peut faire un demi-tour puis tourner dans le sens anti-horaire)

Cela engendre juste le problème si $t_{max} = 1$, mais ce problème sera ignoré par la suite car il rajouterait du temps de calcul pour assez peu de pertinence.

In [14]:
def generate_constraint_clauses(actions: set[tuple[Face, Direction, int]]) -> list[NamedClause]:
    """
    Génère les clauses pour les contraintes.
    """
    clauses: list[NamedClause] = []

    for t in range(1, Variable.t_max + 1):
        clauses.append(
            (
                f"Action obligatoire à chaque étape, temps {t}",
                [Var.Actions(f, d, depth, t) for f, d, depth in actions],
            )
        )

        for f, d, depth in actions:
            action = Var.Actions(f, d, depth, t)

            for f_prime, d_prime, depth_prime in actions:
                if (f, d, depth) < (f_prime, d_prime, depth_prime):
                    clauses.append(
                        (
                            f"Interdiction de rotations multiples, temps {t}, face {f}, {f_prime} et direction {d}, {d_prime}",
                            [
                                -action,
                                -Var.Actions(f_prime, d_prime, depth_prime, t),
                            ],
                        )
                    )

    return clauses

#### <b> 6.4 Clauses liées aux conditions d'arrêt </b>

<b> Positions finales : </b>

L'$id$ étant définie en fonction de la position finale on doit avoir :
$\forall \ c \in C :$

En notant $c = c^0 c^1 c^2$ sa représentation binaire :
$$c^0 \odot x_{c, 0}(t_{max}) \wedge c^1 \odot x_{c, 1}(t_{max}) \wedge c^2 \odot x_{c, 2}(t_{max})$$

<b> Orientations finales : </b>

Les orientations par défaut étant $o = 0$ on doit avoir :
$$\forall \ c \in C, \quad \theta_{c, 0}(t_{max})$$

In [15]:
def generate_final_clauses_x(x: type[VariableX]) -> list[NamedClause]:
    clauses: list[NamedClause] = []

    for pos in x.pos_range():
        for var in x.from_decoded(pos, x.pos_to_idx(pos), Variable.t_max):
            clauses.append(
                (f"Etat final {x.__qualname__} x, position du cube {pos}", [var])
            )

    return clauses


def generate_final_clauses_theta(theta: type[VariableTheta]) -> list[NamedClause]:
    clauses: list[NamedClause] = []

    for pos in theta.pos_range():
        clauses.append(
            (
                f"Etat final {theta.__qualname__} x, position du cube {pos}",
                [theta(pos, 0, Variable.t_max)],  # type: ignore
            )
        )

    return clauses

def generate_final_clauses() -> list[NamedClause]:
    """
    Génère les clauses finales qui définissent l'état final du Rubik's Cube.
    """
    return generate_final_clauses_x(Var.Corners.x) + generate_final_clauses_theta(Var.Corners.theta)

In [16]:
import subprocess

from tqdm import tqdm


class RubiksCubeSolver:
    def __init__(
        self,
        cnf_filename="rubiks_cube.cnf",
    ):
        self.cnf_filename = cnf_filename
    

    def generate_clauses(self, cube: RubiksCube, t_max: int) -> list[NamedClause]:
        """
        Génère toutes les clauses du problème :
        - Les conditions initiales
        - Les règles de transition
        - Les conditions d'arrêt (état final)
        """
        Variable.t_max = t_max
        
        clauses: list[NamedClause] = []
        
        actions = {*product(Var.faces, Var.directions, Var.depths)}
        
        clauses += generate_initial_clauses(cube)
        clauses += generate_all_transitions_clauses(actions)
        clauses += generate_constraint_clauses(actions)
        clauses += generate_final_clauses()

        return clauses

    def generate_str(self, clauses: list[Clause]) -> str:
        """
        Génère une chaîne de caractères pour les clauses.
        """
        s = f"p cnf {Var.n_vars()} {len(clauses)}\n"
        for clause in clauses:
            s += " ".join(map(lambda var: var.id_repr(), clause)) + " 0\n"
        return s

    def generate_cnf_file(self, clauses: list[Clause]) -> None:
        """
        Génère le fichier CNF pour le problème.
        """
        with open(self.cnf_filename, "w") as f:
            f.write(self.generate_str(clauses))

    def verify(
        self, vars: list[Variable], clauses: list[NamedClause]
    ) -> tuple[bool, list[NamedClause]]:
        """
        Vérifie si les variables sont satisfaisantes pour les clauses.

        Retourne un tuple (booléen, liste de clauses insatisfaites).
        
        Utilisé pour le débogage.
        """
        unsat_clauses: list[NamedClause] = []
        var_map = {var.id: var.is_true for var in vars}

        for clause in clauses:
            if not any(var_map.get(var.id, False) == var.is_true for var in clause[1]):
                unsat_clauses.append(clause)

        return len(unsat_clauses) == 0, unsat_clauses

    def solve(
        self, clauses: list[Clause]
    ) -> tuple[bool, list[Variable], list[Var.Actions]]:
        """
        Exécute Gophersat et récupère le résultat.
        """
        self.generate_cnf_file(clauses)

        result = subprocess.run(
            ["gophersat", "--verbose", self.cnf_filename],
            capture_output=True,
            text=True,
        )

        return self.parse_output(result.stdout)

    def parse_output(
        self, output: str
    ) -> tuple[bool, list[Variable], list[Var.Actions]]:
        """
        Analyse la sortie de Gophersat et retourne les actions et positions trouvées.
        """
        if "UNSATISFIABLE" in output:
            return False, [], []

        variables: list[Variable] = []

        for line in output.splitlines():
            if line.startswith("v "):  # Ligne contenant les variables SAT
                values = map(int, line[2:].strip().split())
                variables.extend([Var.from_int(v) for v in values if v > 0])

        actions: list[Var.Actions] = [
            a for a in variables if isinstance(a, Var.Actions)
        ]

        return (
            True,
            variables,
            [action for action in sorted(actions, key=lambda a: a.t)],
        )

    def remove_duplicates(self, actions: list[Var.Actions]) -> list[Var.Actions]:
        """Remove any useless action on a list of actions (e.g. two consecutive actions on the same face)"""
        new_actions: list[Var.Actions] = []

        for action in actions:
            if len(new_actions) == 0:
                new_actions.append(
                    Var.Actions(
                        action.face,
                        action.direction,
                        action.depth,
                        len(new_actions) + 1,
                    )
                )
                continue

            prev_action = new_actions[-1]

            if action.face == prev_action.face and action.depth == prev_action.depth:
                new_direction = action.direction + prev_action.direction

                if new_direction == Direction.NONE:
                    new_actions.pop()

                else:
                    new_actions[-1] = Var.Actions(
                        action.face, new_direction, action.depth, len(new_actions) + 1
                    )
            else:
                new_actions.append(
                    Var.Actions(
                        action.face,
                        action.direction,
                        action.depth,
                        len(new_actions) + 1,
                    )
                )

        return new_actions

    def run(self, cube: RubiksCube, t_max: int) -> tuple[bool, list[Var.Actions]]:
        """
        Génère le CNF, exécute Gophersat et récupère le résultat.
        """
        clauses = self.generate_clauses(cube, t_max)
        sat, variables, actions = self.solve(self.remove_name(clauses))

        return sat, actions

    def find_optimal(self, cube: RubiksCube) -> list[Var.Actions]:
        """
        Trouve la solution optimale en testant différents `t_max`.
        """
        cube = cube.copy()
        actions: list[Var.Actions] = []
        
        Variable.cube_size = cube.size
        Var.depths = list(range(Variable.cube_size - 1))

        sat_found: bool = False
        unsat_found: bool = False
        t = Variable.t_max // 2  # On commence au milieu

        actions_this_run: list[Var.Actions] = []

        while not sat_found or not unsat_found:
            sat, actions_ = self.run(cube, t)

            print(f"t = {t}, sat = {sat}")

            if sat:
                sat_found = True
                actions_this_run = actions_
                t -= 1
            else:
                unsat_found = True
                t += 1

        for action in actions_this_run:
            cube.rotate(action.face, action.direction, action.depth)

        actions += actions_this_run

        return self.remove_duplicates(actions)

    def remove_name(self, clauses: list[NamedClause]) -> list[Clause]:
        return [clause[1] for clause in clauses]


### <b> 7. Résolution du Rubik's Cube </b>

In [17]:

Variable.cube_size = 2

rubiks_cube = RubiksCube(2)
rubiks_cube.shuffle(replace_origin=True) # Replace origin so that the cube of id 0 is at the top front left corner

solver = RubiksCubeSolver("rubiks_cube.cnf")
actions = solver.find_optimal(rubiks_cube)

print(f"Solved in {len(actions)} moves")

moves = [
    RubiksCube.move_to_str(action.face, action.direction, action.depth)
    for action in actions
]

rubiks_cube.animate(RubiksCube.parse_moves(moves), speed=5)


t = 5, sat = False
t = 6, sat = False


t = 7, sat = False
t = 8, sat = False
t = 9, sat = True
Solved in 9 moves


<h1 align="left"> <font color='deepskyblue'> Partie 2 : Rubik's Cube 3×3×3 </font></a></h1>  

### <b> 1. Remarques </b>

Le **Rubik’s Cube 3×3×3** est une généralisation du Rubik's Cube 2×2×2, avec l'apparition d'autres catégories de cubes autres que les coins : les cubes des centres et les cubes des arêtes.

<p align="center">
  <img src="images/RubiksCube3CentresAretes.png" alt="RubiksCube3CentresAretes" width="700">
</p>

Les cubes des arêtes ont 2 orientations possibles 0 et 1, et les cubes des centres n'en ont qu'une seule, nous ne l'implémentons donc pas.

In [18]:
EdgePos = int
EdgeIdx = Literal[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
CenterPos = int
CenterIdx = Literal[0, 1, 2, 3, 4, 5]
EdgeOrientation = Literal[0, 1]
Size = tuple[int, int, int]

On doit également ajouter des actions pour pouvoir résoudre les Rubik's Cube.
Pour conserver le coin fixe on a décider pour chaque action prédédente d'ajouter une profondeur (`depth`) c'est à dire qu'on peut maintenant tourner la face de droite dans le sens horaire comme avant mais on peut également tourner la même face avec $depth = 1$ (au lieu de 0) pour tourner le milieu du Rubik's Cube (on peut facilement généraliser ce concept pour des Rubik's Cube de taille $n \times n \times n$)

### <b> 2. Ajout des variables liées aux centres et aux arêtes </b>

#### 2.1. Classes abstraites

Pour rendre les classes VariableX, VariableTheta et VariableState indépendantes du type de cube, nous définissons deux variables de type générique `TPos`,  et 
`TIdx` :

- `TPos` représente la position d'un cube dans l'espace, et peut être CornerPos, EdgePos ou CenterPos. 
- `TIdx` représente l'identifiant ou l'orientation d'un cube à une position donnée, et peut être CornerPos, EdgePos, CenterPos, CornerOrientation ou EdgeOrientation. 



In [19]:
from typing import Generic, Iterable, TypeVar


TPos = TypeVar("TPos", CornerPos, EdgePos, CenterPos)
TIdx = TypeVar(
    "TIdx", CornerPos, EdgePos, CenterPos, CornerOrientation, EdgeOrientation
)


class VariableParent(ABC, Generic[TPos]):
    @classmethod
    @abstractmethod
    def n_vars(cls) -> int: ...

    @classmethod
    @abstractmethod
    def g(cls, pos: TPos) -> tuple[int, int, int]: ...

    @classmethod
    @abstractmethod
    def g_inv(cls, c_x: int, c_y: int, c_z: int) -> TPos: ...

    @classmethod
    @abstractmethod
    def n_pos(cls) -> int: ...

    @classmethod
    def pos_range(cls) -> Iterable[TPos]:
        return range(cls.n_pos())  # type: ignore


class VariableState(Variable, Generic[TPos, TIdx], ABC):
    def __init__(self, pos: TPos, idx: TIdx, t: int, is_true: bool = True) -> None:
        self.pos: TPos = pos
        self.idx: TIdx = idx

        super().__init__(t, is_true)

    @classmethod
    @abstractmethod
    def parent(cls) -> type[VariableParent[TPos]]: ...

    @classmethod
    @abstractmethod
    def n_idx(cls) -> int: ...

    @classmethod
    def idx_range(cls) -> Iterable[TIdx]:
        return range(cls.n_idx())  # type: ignore

    @classmethod
    def n_pos(cls) -> int:
        return cls.parent().n_pos()

    @classmethod
    def pos_range(cls) -> Iterable[TPos]:
        return cls.parent().pos_range()

    @classmethod
    def from_int(cls, var: int) -> "VariableState[TPos, TIdx]":
        var -= cls.offset()
        n_pos = cls.n_pos()
        n_idx = cls.n_idx()

        return cls(
            var % n_pos,  # type: ignore
            (var // n_pos) % n_idx,  # type: ignore
            (var // (n_idx * n_pos)) % (Variable.t_max + 1),
        )

    @classmethod
    def n_vars(cls) -> int:
        return cls.n_pos() * cls.n_idx() * (Variable.t_max + 1)

    def compute_id(self) -> int:
        return (
            self.offset()
            + self.pos
            + self.idx * self.n_pos()
            + self.t * self.n_idx() * self.n_pos()
        )

    def g(self, pos: TPos) -> tuple[int, int, int]:
        return self.parent().g(pos)

    def g_inv(self, c_x: int, c_y: int, c_z: int) -> TPos:
        return self.parent().g_inv(c_x, c_y, c_z)

    @abstractmethod
    def rotate(
        self, face: Face, direction: Direction, depth: int
    ) -> "VariableState[TPos, TIdx]": ...


class VariableX(VariableState[TPos, TIdx], Generic[TPos, TIdx]):
    def rotate(
        self, face: Face, direction: Direction, depth: int
    ) -> "VariableX[TPos, TIdx]":
        return self.__class__(
            self.g_inv(*rotate_cube(*self.g(self.pos), face, direction, depth)),
            self.idx,
            self.t + 1,
        )

    @classmethod
    def encode(cls, decoded_idx: int) -> tuple[int, ...]:
        return tuple(
            1 if s == "1" else -1
            for s in np.binary_repr(decoded_idx, width=cls.n_idx())
        )

    @classmethod
    def from_decoded(
        cls, pos: TPos, idx_decoded: TIdx, t: int
    ) -> tuple["VariableState[TPos, TIdx]", ...]:
        return tuple(
            sign * cls(pos, cast(TIdx, idx), t)
            for idx, sign in enumerate(cls.encode(idx_decoded))
        )  # type: ignore

    @classmethod
    @abstractmethod
    def pos_to_idx(cls, pos: TPos) -> TIdx: ...


class VariableTheta(VariableState[TPos, TIdx], Generic[TPos, TIdx]): ...


#### 2.2. Classes concrètes

Nous devons implémenter concrètement les classes `Edges` et `Centers`, sous-classes de `Var`. 

Pour chacune de ces classes, nous ajoutons la sous-classe `x`, et les méthodes `n_vars`, `g` et `g_inv`. 

Nous implémentons également la sous-classe `theta` de la classe `Edges`. Inutile de l'implémenter pour la classe `Centers`, les cubes du centre n'ayant qu'une seule position invariante aux rotations. 

In [20]:
class Var:
    faces = [Face.RIGHT, Face.BOTTOM, Face.BACK]
    directions = [Direction.CLOCKWISE, Direction.HALF_TURN, Direction.COUNTERCLOCKWISE]
    depths: list[int]

    class Corners(VariableParent[CornerPos]):
        class x(VariableX[CornerPos, CornerPos]):
            @classmethod
            def offset(cls) -> int:
                return 1

            @classmethod
            def parent(cls) -> type["Var.Corners"]:
                return Var.Corners

            @classmethod
            def n_idx(cls) -> int:
                return 3

            @classmethod
            def pos_to_idx(cls, pos: CornerPos) -> CornerPos:
                return pos

        class theta(VariableTheta[CornerPos, CornerOrientation]):
            @classmethod
            def n_idx(cls) -> int:
                return 3

            @classmethod
            def offset(cls) -> int:
                return 1 + Var.Corners.x.n_vars()

            @classmethod
            def parent(cls) -> type["Var.Corners"]:
                return Var.Corners

            def rotate(
                self,
                face: Face,
                direction: Direction,
                depth: int,
            ) -> "Var.Corners.theta":
                c_x, c_y, c_z = self.g(self.pos)

                if not will_rotate(c_x, c_y, c_z, face, depth):
                    return Var.Corners.theta(self.pos, self.idx, self.t + 1)

                new_pos = self.g_inv(
                    *rotate_cube(c_x, c_y, c_z, face, direction, depth)
                )

                if direction == Direction.HALF_TURN:
                    return Var.Corners.theta(new_pos, self.idx, self.t + 1)

                def s(
                    i: CornerOrientation,
                    j: CornerOrientation,
                    orientation: CornerOrientation,
                ) -> CornerOrientation:
                    if orientation == i:
                        return j
                    if orientation == j:
                        return i
                    return orientation

                if face == Face.RIGHT:
                    new_orientation = s(0, 2, self.idx)
                elif face == Face.BOTTOM:
                    new_orientation = s(0, 1, self.idx)
                elif face == Face.BACK:
                    new_orientation = s(1, 2, self.idx)
                else:
                    raise ValueError(f"Invalid face: {face}")

                return Var.Corners.theta(new_pos, new_orientation, self.t + 1)

        @classmethod
        def n_vars(cls) -> int:
            return Var.Corners.x.n_vars() + Var.Corners.theta.n_vars()

        @classmethod
        def g(cls, pos: CornerPos) -> tuple[int, int, int]:
            return (
                (pos % 2) * (Variable.cube_size - 1),
                ((pos // 2) % 2) * (Variable.cube_size - 1),
                ((pos // 4) % 2) * (Variable.cube_size - 1),
            )

        @classmethod
        def g_inv(cls, c_x: int, c_y: int, c_z: int) -> CornerPos:
            return cast(
                CornerPos,
                int(c_x != 0) + 2 * int(c_y != 0) + 4 * int(c_z != 0),
            )

        @classmethod
        def n_pos(cls) -> int:
            return 8

        @classmethod
        def pos_range(cls) -> Iterable[CornerPos]:
            return range(
                1, Var.Corners.n_pos()
            )  # Exclude origin (because it does not move) # type: ignore

    class Edges(VariableParent[EdgePos]):
        class x(VariableX[EdgePos, EdgeIdx]):
            @classmethod
            def offset(cls) -> int:
                return 1 + Var.Corners.n_vars()

            @classmethod
            def parent(cls) -> type["Var.Edges"]:
                return Var.Edges

            @classmethod
            def n_idx(cls) -> int:
                if Variable.cube_size <= 2:
                    return 0
                return 4

            @classmethod
            def pos_to_idx(cls, pos: EdgePos) -> EdgeIdx:
                x, y, z = Var.Edges.g(pos)
                axis = np.argmax(
                    [0 < p < Variable.cube_size - 1 for p in [x, y, z]]
                ).item()
                other_coords = [
                    0 if p == 0 else 1 for i, p in enumerate([x, y, z]) if i != axis
                ]
                return cast(EdgeIdx, 4 * axis + 2 * other_coords[1] + other_coords[0])

        class theta(VariableTheta[EdgePos, EdgeOrientation]):
            @classmethod
            def n_idx(cls) -> int:
                return 1

            @classmethod
            def offset(cls) -> int:
                return 1 + Var.Corners.n_vars() + Var.Edges.x.n_vars()

            @classmethod
            def parent(cls) -> type["Var.Edges"]:
                return Var.Edges

            def rotate(
                self,
                face: Face,
                direction: Direction,
                depth: int,
            ) -> "Var.Edges.theta":
                c_x, c_y, c_z = self.g(self.pos)

                if not will_rotate(c_x, c_y, c_z, face, depth):
                    return Var.Edges.theta(self.pos, self.idx, self.t + 1)

                new_pos = self.g_inv(
                    *rotate_cube(c_x, c_y, c_z, face, direction, depth)
                )

                if direction == Direction.HALF_TURN:
                    return Var.Edges.theta(new_pos, 0, self.t + 1, self.is_true)

                if face != Face.RIGHT and depth == 0:
                    return Var.Edges.theta(new_pos, 0, self.t + 1, self.is_true)

                return Var.Edges.theta(new_pos, 0, self.t + 1, not self.is_true)

        @classmethod
        def n_vars(cls) -> int:
            return Var.Edges.x.n_vars() + Var.Edges.theta.n_vars()

        @classmethod
        def g(cls, pos: EdgePos) -> tuple[int, int, int]:
            axis_pos = (pos % (Variable.cube_size - 2)) + 1

            pos_ = pos // (Variable.cube_size - 2)
            other_coords = [
                (pos_ % 2) * (Variable.cube_size - 1),
                ((pos_ // 2) % 2) * (Variable.cube_size - 1),
            ]

            axis = pos_ // 4
            other_coords.insert(axis, axis_pos)

            return other_coords[0], other_coords[1], other_coords[2]

        @classmethod
        def g_inv(cls, c_x: int, c_y: int, c_z: int) -> EdgePos:
            coords = [c_x, c_y, c_z]

            axis = np.argmax([0 < p < Variable.cube_size - 1 for p in coords]).item()
            other_coords = [int(p != 0) for i, p in enumerate(coords) if i != axis]

            return cast(
                EdgePos,
                (other_coords[0] + other_coords[1] * 2 + axis * 4)
                * (Variable.cube_size - 2)
                + coords[axis]
                - 1,
            )

        @classmethod
        def n_pos(cls) -> int:
            return 12 * (Variable.cube_size - 2)

    class Centers(VariableParent[CenterPos]):
        class x(VariableX[CenterPos, CenterIdx]):
            @classmethod
            def offset(cls) -> int:
                return 1 + Var.Corners.n_vars() + Var.Edges.n_vars()

            @classmethod
            def parent(cls) -> type["Var.Centers"]:
                return Var.Centers

            @classmethod
            def n_idx(cls) -> int:
                if Variable.cube_size <= 2:
                    return 0
                return 3

            @classmethod
            def pos_to_idx(cls, pos: CenterPos) -> CenterIdx:
                x, y, z = Var.Centers.g(pos)
                axis = np.argmax(
                    [p == 0 or p == Variable.cube_size - 1 for p in [x, y, z]]
                ).item()
                axis_pos = 0 if [x, y, z][axis] == 0 else 1

                return cast(CenterIdx, 2 * axis + axis_pos)

        @classmethod
        def n_vars(cls) -> int:
            return Var.Centers.x.n_vars()

        @classmethod
        def g(cls, pos: CenterPos) -> tuple[int, int, int]:
            gamma = (Variable.cube_size - 2) ** 2
            alpha = pos // gamma
            beta = pos % gamma
            axis = alpha // 2
            i = alpha % 2

            a = 1 + beta % (Variable.cube_size - 2)
            b = 1 + beta // (Variable.cube_size - 2)
            pos_axis = i * (Variable.cube_size - 1)

            coordinates = [a, b]
            coordinates.insert(axis, pos_axis)

            c_x, c_y, c_z = coordinates[0], coordinates[1], coordinates[2]

            return c_x, c_y, c_z

        @classmethod
        def g_inv(cls, c_x: int, c_y: int, c_z: int) -> CenterPos:
            gamma = (Variable.cube_size - 2) ** 2
            x_0 = 0
            x_max = gamma
            y_0 = 2 * gamma
            y_max = 3 * gamma
            z_0 = 4 * gamma
            z_max = 5 * gamma

            center_pos = 0

            if c_x == 0:
                center_pos = x_0 + (c_y - 1) + (Variable.cube_size - 2) * (c_z - 1)

            elif c_x == Variable.cube_size - 1:
                center_pos = x_max + (c_y - 1) + (Variable.cube_size - 2) * (c_z - 1)

            elif c_y == 0:
                center_pos = y_0 + (c_x - 1) + (Variable.cube_size - 2) * (c_z - 1)

            elif c_y == Variable.cube_size - 1:
                center_pos = y_max + (c_x - 1) + (Variable.cube_size - 2) * (c_z - 1)

            elif c_z == 0:
                center_pos = z_0 + (c_x - 1) + (Variable.cube_size - 2) * (c_y - 1)

            elif c_z == Variable.cube_size - 1:
                center_pos = z_max + (c_x - 1) + (Variable.cube_size - 2) * (c_y - 1)

            return cast(CenterPos, center_pos)

        @classmethod
        def n_pos(cls) -> int:
            return 6 * (Variable.cube_size - 2) ** 2

    class Actions(Variable):
        def __init__(
            self,
            face: Face,
            direction: Direction,
            depth: int,
            t: int,
            is_true: bool = True,
        ):
            self.face = face
            self.direction = direction
            self.depth = depth

            super().__init__(t, is_true)

        def compute_id(self) -> int:
            return (
                self.offset()
                + Var.directions.index(self.direction)
                + Var.faces.index(self.face) * 3
                + self.depth * 9
                + self.t * 9 * (Variable.cube_size - 1)
            )

        @classmethod
        def offset(cls) -> int:
            return 1 + Var.Corners.n_vars() + Var.Edges.n_vars() + Var.Centers.n_vars()

        @classmethod
        def n_vars(cls) -> int:
            return 9 * (Variable.t_max + 1) * (Variable.cube_size - 1)

        @classmethod
        def from_int(cls, var: int) -> "Var.Actions":
            var -= cls.offset()

            return Var.Actions(
                Var.faces[(var // 3) % 3],
                Var.directions[var % 3],
                (var // 9) % (Variable.cube_size - 1),
                (var // (9 * (Variable.cube_size - 1))),
            )

        def get_params(self) -> tuple[Face, Direction, int, int]:
            return self.face, self.direction, self.depth, self.t

    @staticmethod
    def n_vars() -> int:
        return (
            Var.Corners.n_vars()
            + Var.Edges.n_vars()
            + Var.Centers.n_vars()
            + Var.Actions.n_vars()
        )

    @staticmethod
    def from_int(var: int) -> Variable:
        var_class: list[type[Variable]] = [
            Var.Corners.x,
            Var.Corners.theta,
            Var.Edges.x,
            Var.Edges.theta,
            Var.Centers.x,
            Var.Actions,
        ]

        lower_bound = 1
        for cls in var_class:
            n_vars = cls.n_vars()

            if lower_bound <= var < lower_bound + n_vars:
                return cls.from_int(var)
            lower_bound += n_vars

        raise ValueError(f"Invalid variable: {var}")


### <b> 3. Implémentation des clauses en plusieurs étapes</b>

On s'est rendu compte qu'essayer de résoudre le rubiks cube d'un coup prenait beaucoup trop de temps.

Afin d'optimiser le temps de calcul du solver, on découpe notre problème en plusieurs plus petit problèmes inspirée de méthodes classiques de résolution humaine d'un Rubik's Cube 3×3×3.

Pour certaines étapes on peut même restreindre l'ensemble d'actions possibles pour améliorer la rapidité.

Les étapes sont : 

**1. Coins :**

Placement et orientation des coins → `Corners`

Clauses finales associées :
- Position : 
  $$\forall \ c \in Corners, \quad x_{c, c}(t_{max})$$  
- Orientation :
  $$\forall \ c \in Corners, \quad \theta_{c, 0}(t_{max})$$

Actions :
- On peut restreindre l'ensemble des actions uniquement à celle où $depth = 0$

**2. Centres et Orientation des arrêtes :** 

2.1 Orientations des arrêtes → `EdgeOrientation`

2.2 Placement des centres  → `Centers`

Clauses finales associées :
- Position : 
  $$\forall \ c \in Centers, \quad x_{c, c}(t_{max})$$
- Orientation : 
  $$\forall \ c \in Edges, \quad \theta_{c, 0}(t_{max})$$
  $$\forall \ c \in Centers, \quad \theta_{c, 0}(t_{max})$$

**3. Position des arrêtes sur leur cercle :**  

Placement des cubes des arêtes sur le bon cercle → `EdgePositionOnCircle`

Un cercle correspond à un ensemble de cubes des arêtes présents sur une même face : 

<p align="center">
  <img src="images/RubiksCube3Cercle.png" alt="RubiksCube3Cercle" width="300">
</p>

Clauses finales associées :
- Position :  


**4. Position des arrêtes dans les cercles :**

Placement des cubes des arêtes d'un cercle à la bonne place dans le cercle → `EdgePosition`

Clauses finales associées :
- Position : 



In [21]:
class Step(ABC):
    def actions(self) -> set[tuple[Face, Direction, int]]:
        return  {*product(Var.faces, Var.directions, Var.depths)}

    def t_median(self) -> int:
        return 11

    @abstractmethod
    def generate_final_clauses(self) -> list[NamedClause]: ...

    def __add__(self, other: "Step") -> "Step":
        return Combined(self, other)

    def __repr__(self) -> str:
        return self.__class__.__name__ + "()"


class Combined(Step):
    """Superposition de plusieurs étapes."""
    def __init__(self, *steps: Step):
        self.steps = steps
    
    def actions(self) -> set[tuple[Face, Direction, int]]:
        return set.union(*(step.actions() for step in self.steps))

    def t_median(self) -> int:
        return round(np.median([step.t_median() for step in self.steps]))

    def generate_final_clauses(self) -> list[NamedClause]:
        return [
            clause for step in self.steps for clause in step.generate_final_clauses()
        ]

    def __repr__(self) -> str:
        return " + ".join(repr(step) for step in self.steps)


class Corners(Step):
    def actions(self) -> set[tuple[Face, Direction, int]]:
        return {*product(Var.faces, Var.directions, [0])}

    def t_median(self) -> int:
        return 8

    def generate_final_clauses(self) -> list[NamedClause]:
        return generate_final_clauses_x(Var.Corners.x) + generate_final_clauses_theta(
            Var.Corners.theta
        )


class Centers(Step):
    def t_median(self) -> int:
        return 2

    def generate_final_clauses(self) -> list[NamedClause]:
        return generate_final_clauses_x(Var.Centers.x)


class EdgeOrientation(Step):
    def generate_final_clauses(self) -> list[NamedClause]:
        clauses: list[NamedClause] = []
        for pos in Var.Edges.pos_range():
            clauses.append(
                (
                    f"Etat final EdgeOrientation theta, edge {pos}",
                    [Var.Edges.theta(pos, 0, Variable.t_max)],
                )
            )
        return clauses


class EdgePositionOnCircle(Step):
    def actions(self) -> set[tuple[Face, Direction, int]]:
        return {*product(Var.faces, [Direction.HALF_TURN], [1])} | {
            *product(
                [Face.BACK, Face.BOTTOM],
                [Direction.CLOCKWISE, Direction.COUNTERCLOCKWISE],
                [0],
            )
        }

    def t_median(self) -> int:
        return 11

    def generate_final_clauses(self) -> list[NamedClause]:
        clauses: list[NamedClause] = []
        for pos in [0, 1, 2, 3]:
            clauses.append(
                (
                    f"Etat final EdgePosition x, edge {pos}",
                    [-Var.Edges.x(pos, 0, Variable.t_max)],
                )
            )
            clauses.append(
                (
                    f"Etat final EdgePosition x, edge {pos}",
                    [-Var.Edges.x(pos, 1, Variable.t_max)],
                )
            )

        for pos in [4, 5, 6, 7]:
            clauses.append(
                (
                    f"Etat final EdgePosition x, edge {pos}",
                    [-Var.Edges.x(pos, 0, Variable.t_max)],
                )
            )
            clauses.append(
                (
                    f"Etat final EdgePosition x, edge {pos}",
                    [Var.Edges.x(pos, 1, Variable.t_max)],
                )
            )

        for pos in [8, 9, 10, 11]:
            clauses.append(
                (
                    f"Etat final EdgePosition x, edge {pos}",
                    [Var.Edges.x(pos, 0, Variable.t_max)],
                )
            )
            clauses.append(
                (
                    f"Etat final EdgePosition x, edge {pos}",
                    [-Var.Edges.x(pos, 1, Variable.t_max)],
                )
            )

        return clauses


class EdgePosition(Step):
    def actions(self) -> set[tuple[Face, Direction, int]]:
        return {*product(Var.faces, [Direction.HALF_TURN], [0])} | {
            *product(Var.faces, Var.directions, [1])
        }

    def t_median(self) -> int:
        return 8

    def generate_final_clauses(self) -> list[NamedClause]:
        clauses: list[NamedClause] = []
        for pos in Var.Edges.pos_range():
            for var in Var.Edges.x.from_decoded(
                pos, Var.Edges.x.pos_to_idx(pos), Variable.t_max
            ):
                clauses.append((f"Etat final EdgePosition x, edge {pos}", [var]))

        return clauses

In [22]:
def get_vars_from_edge_pos(cube: RubiksCube, pos: EdgePos) -> tuple[EdgeIdx, EdgeOrientation]:
    coords = list(Var.Edges.g(pos))

    axis = np.argmax([0 < p < cube.size - 1 for p in coords]).item()
    for i in range(len(coords)):
        if i != axis:
            coords[i] = -int(coords[i] != 0)

    match coords:
        case [x, 0, 0]:
            colors = [
                Color(cube.faces[Face.TOP][x, -1]),
                Color(cube.faces[Face.FRONT][x, 0]),
            ]
        case [x, -1, 0]:
            colors = [
                Color(cube.faces[Face.BOTTOM][x, 0]),
                Color(cube.faces[Face.FRONT][x, -1]),
            ]
        case [x, 0, -1]:
            colors = [
                Color(cube.faces[Face.TOP][x, 0]),
                Color(cube.faces[Face.BACK][-x - 1, 0]),
            ]
        case [x, -1, -1]:
            colors = [
                Color(cube.faces[Face.BOTTOM][x, -1]),
                Color(cube.faces[Face.BACK][-x - 1, -1]),
            ]
        case [0, y, 0]:
            colors = [
                Color(cube.faces[Face.LEFT][-1, y]),
                Color(cube.faces[Face.FRONT][0, y]),
            ]
        case [-1, y, 0]:
            colors = [
                Color(cube.faces[Face.RIGHT][0, y]),
                Color(cube.faces[Face.FRONT][-1, y]),
            ]
        case [0, y, -1]:
            colors = [
                Color(cube.faces[Face.LEFT][0, y]),
                Color(cube.faces[Face.BACK][-1, y]),
            ]
        case [-1, y, -1]:
            colors = [
                Color(cube.faces[Face.RIGHT][-1, y]),
                Color(cube.faces[Face.BACK][0, y]),
            ]
        case [0, 0, z]:
            colors = [
                Color(cube.faces[Face.TOP][0, -z - 1]),
                Color(cube.faces[Face.LEFT][-z - 1, 0]),
            ]
        case [-1, 0, z]:
            colors = [
                Color(cube.faces[Face.TOP][-1, -z - 1]),
                Color(cube.faces[Face.RIGHT][z, 0]),
            ]
        case [0, -1, z]:
            colors = [
                Color(cube.faces[Face.BOTTOM][0, z]),
                Color(cube.faces[Face.LEFT][-z - 1, -1]),
            ]
        case [-1, -1, z]:
            colors = [
                Color(cube.faces[Face.BOTTOM][-1, z]),
                Color(cube.faces[Face.RIGHT][z, -1]),
            ]
        case _:
            raise ValueError(f"Invalid edge position: {pos}")

    orientation = cast(EdgeOrientation, int(colors[0].value < colors[1].value))
    idx = None
    # X Axis
    if Color.BLUE not in colors and Color.GREEN not in colors:
        idx = int(Color.ORANGE in colors) + 2 * int(Color.YELLOW in colors)
    # Y Axis
    if Color.RED not in colors and Color.ORANGE not in colors:
        idx = 4 + int(Color.GREEN in colors) + 2 * int(Color.YELLOW in colors)
    # Z Axis
    if Color.WHITE not in colors and Color.YELLOW not in colors:
        idx = 8 + int(Color.GREEN in colors) + 2 * int(Color.ORANGE in colors)

    assert idx is not None, f"Invalid edge colors: {colors}"

    return cast(EdgeIdx, idx), orientation

def get_vars_from_center_pos(cube: RubiksCube, pos: CenterPos) -> CenterIdx:
    x, y, z = Var.Centers.g(pos)

    color = None
    if x == 0:
        color = Color(cube.faces[Face.LEFT][-z - 1, y])
    if x == cube.size - 1:
        color = Color(cube.faces[Face.RIGHT][z, y])
    if y == 0:
        color = Color(cube.faces[Face.TOP][x, -z - 1])
    if y == cube.size - 1:
        color = Color(cube.faces[Face.BOTTOM][x, z])
    if z == 0:
        color = Color(cube.faces[Face.FRONT][x, y])
    if z == cube.size - 1:
        color = Color(cube.faces[Face.BACK][-x - 1, y])
    assert color is not None, f"Invalid center position: {pos}"

    return cast(
        CenterIdx,
        {
            Color.BLUE: 0,
            Color.GREEN: 1,
            Color.RED: 2,
            Color.ORANGE: 3,
            Color.WHITE: 4,
            Color.YELLOW: 5,
        }[color],
    )

In [23]:
def generate_initial_clauses(cube: RubiksCube) -> list[NamedClause]:
    clauses: list[NamedClause] = []

    for pos in Var.Corners.pos_range():
        idx, theta = get_vars_from_corner_pos(cube, pos)

        for var in Var.Corners.x.from_decoded(pos, idx, 0):
            clauses.append(("Initial state position", [var]))

        for orientation in Var.Corners.theta.idx_range():
            clauses.append(
                (
                    "Initial state orientation",
                    [Var.Corners.theta(pos, orientation, 0, orientation == theta)],
                )
            )

    for pos in Var.Edges.pos_range():
        idx, theta = get_vars_from_edge_pos(cube, pos)

        for var in Var.Edges.x.from_decoded(pos, idx, 0):
            clauses.append(("Initial state position", [var]))

        clauses.append(
            ("Initial state orientation", [Var.Edges.theta(pos, 0, 0, bool(theta))])
        )

    for pos in Var.Centers.pos_range():
        idx = get_vars_from_center_pos(cube, pos)

        for var in Var.Centers.x.from_decoded(pos, idx, 0):
            clauses.append(("Initial state position", [var]))

    return clauses

def generate_all_transitions_clauses(actions: set[tuple[Face, Direction, int]]) -> list[NamedClause]:
    """
    Génère les clauses pour les transitions.
    """
    clauses: list[NamedClause] = []

    for t in range(1, Variable.t_max + 1):
        for f, d, depth in actions:
            action = Var.Actions(f, d, depth, t)

            clauses += generate_transitions_clauses_for(Var.Corners.x, action)
            clauses += generate_transitions_clauses_for(Var.Corners.theta, action)
            clauses += generate_transitions_clauses_for(Var.Edges.x, action)
            clauses += generate_transitions_clauses_for(Var.Edges.theta, action)
            clauses += generate_transitions_clauses_for(Var.Centers.x, action)

    return clauses

def generate_clauses_with_steps(
    cube: RubiksCube, t_max: int, steps: list[Step]
) -> list[NamedClause]:
    Variable.t_max = t_max
    
    actions = steps[-1].actions()

    clauses: list[NamedClause] = []
    clauses += generate_initial_clauses(cube)
    clauses += generate_constraint_clauses(actions)
    clauses += generate_all_transitions_clauses(actions)

    for step in steps:
        clauses += step.generate_final_clauses()

    return clauses

def run_with_steps(
    self, cube: RubiksCube, t_max: int, steps: list[Step]
) -> tuple[bool, list[Var.Actions]]:
    """
    Génère le CNF, exécute Gophersat et récupère le résultat.
    """
    clauses = generate_clauses_with_steps(cube, t_max, steps)
    sat, variables, actions = self.solve(self.remove_name(clauses))

    return sat, actions

RubiksCubeSolver.run_with_steps = run_with_steps

def find_optimal_with_steps(self, cube: RubiksCube, steps: list[Step]) -> list[Var.Actions]:
    """
    Trouve la solution optimale.
    """
    cube = cube.copy()
    actions: list[Var.Actions] = []
    
    Variable.cube_size = cube.size
    Var.depths = list(range(Variable.cube_size - 1))

    for step_idx in tqdm(range(len(steps))):
        step = steps[step_idx]
        sat_found: bool = False
        unsat_found: bool = False
        t = step.t_median()

        actions_this_step: list[Var.Actions] = []

        while not sat_found or not unsat_found:
            sat, actions_ = self.run_with_steps(cube, t, steps[: step_idx + 1])

            print(f"Step {step}, t = {t}, sat = {sat}")

            if sat:
                actions_this_step = actions_
                if t == 0:
                    break

                sat_found = True
                t -= 1

            else:
                unsat_found = True
                t += 1

        for action in actions_this_step:
            cube.rotate(action.face, action.direction, action.depth)

        actions += actions_this_step

    return self.remove_duplicates(actions)

RubiksCubeSolver.find_optimal_with_steps = find_optimal_with_steps


### <b> 4. Résolution du Rubik's Cube

In [24]:
Variable.cube_size = 3

rubiks_cube = RubiksCube(3)
rubiks_cube.shuffle(replace_origin=True)

solver = RubiksCubeSolver("rubiks_cube.cnf")

actions = solver.find_optimal_with_steps(
    rubiks_cube,
    steps=[
        Corners(),
        EdgeOrientation() + Centers(),
        EdgePositionOnCircle(),
        EdgePosition(),
    ],
)

print(f"Solved in {len(actions)} moves")
moves = [
    RubiksCube.move_to_str(action.face, action.direction, action.depth)
    for action in actions
]

rubiks_cube.animate(RubiksCube.parse_moves(moves), speed=5)


  0%|          | 0/4 [00:00<?, ?it/s]

Step Corners(), t = 8, sat = False
Step Corners(), t = 9, sat = False


 25%|██▌       | 1/4 [02:59<08:59, 180.00s/it]

Step Corners(), t = 10, sat = True
Step EdgeOrientation() + Centers(), t = 6, sat = False
Step EdgeOrientation() + Centers(), t = 7, sat = False


 50%|█████     | 2/4 [03:04<02:33, 76.60s/it] 

Step EdgeOrientation() + Centers(), t = 8, sat = True
Step EdgePositionOnCircle(), t = 11, sat = False
Step EdgePositionOnCircle(), t = 12, sat = False
Step EdgePositionOnCircle(), t = 13, sat = False
Step EdgePositionOnCircle(), t = 14, sat = False


 75%|███████▌  | 3/4 [03:41<00:58, 58.71s/it]

Step EdgePositionOnCircle(), t = 15, sat = True
Step EdgePosition(), t = 8, sat = False
Step EdgePosition(), t = 9, sat = False
Step EdgePosition(), t = 10, sat = False


100%|██████████| 4/4 [04:12<00:00, 63.15s/it]

Step EdgePosition(), t = 11, sat = True
Solved in 43 moves


> Remarque : le solveur fonctionne encore pour un Rubik's Cube de taille 2×2×2, il suffit de changer le paramètre size à 2 et mettre comme steps uniquement les coins.

<h1 align="left"> <font color='deepskyblue'> Partie 3 : Rubik's Cube 4×4×4 </font></a></h1>  

Nous rajoutons ces étapes pour la résolution du Rubik's Cube 4×4×4 :  

In [25]:
class WhiteAndYellowCenters(Step):
    def t_median(self) -> int:
        return 7

    def generate_final_clauses(self) -> list[NamedClause]:
        clauses: list[NamedClause] = []

        for pos in Var.Centers.x.pos_range():
            idx = Var.Centers.x.pos_to_idx(pos)

            if idx < 4:
                continue

            for var in Var.Centers.x.from_decoded(pos, idx, Variable.t_max):
                clauses.append((f"Etat final Centers x, position du cube {pos}", [var]))

        return clauses


class WhiteAndYellowCentersLine(Step):
    def __init__(self, line_idx: int) -> None:
        self.line_idx = line_idx

    def t_median(self) -> int:
        return 5

    def generate_final_clauses(self) -> list[NamedClause]:
        clauses: list[NamedClause] = []

        for pos in Var.Centers.x.pos_range():
            idx = Var.Centers.x.pos_to_idx(pos)

            if idx < 4:
                continue

            coords = Var.Centers.g(pos)
            axis = np.argmax([p == 0 or p == Variable.cube_size - 1 for p in coords])
            other_coords = [p for i, p in enumerate(coords) if i != axis]

            if other_coords[0] - 1 != self.line_idx:
                continue

            for var in Var.Centers.x.from_decoded(pos, idx, Variable.t_max):
                clauses.append((f"Etat final Centers x, position du cube {pos}", [var]))

        return clauses


class CentersNotWhiteOrYellow(Step):
    def actions(self) -> set[tuple[Face, Direction, int]]:
        return (
            {*product(Var.faces, [Direction.HALF_TURN], [1, 2])}
            | {
                *product(
                    [Face.BACK],
                    [Direction.CLOCKWISE, Direction.COUNTERCLOCKWISE],
                    [1, 2],
                )
            }
            | {*product([Face.BOTTOM, Face.RIGHT], Var.directions, [0])}
        )

    def t_median(self) -> int:
        return 8

    def generate_final_clauses(self) -> list[NamedClause]:
        clauses: list[NamedClause] = []

        for pos in Var.Centers.x.pos_range():
            idx = Var.Centers.x.pos_to_idx(pos)

            if idx >= 4:
                continue

            for var in Var.Centers.x.from_decoded(pos, idx, Variable.t_max):
                clauses.append((f"Etat final Centers x, position du cube {pos}", [var]))

        return clauses


class SameEdges(Step):
    def __init__(self, edge_idx: EdgeIdx) -> None:
        self.edge_idx = edge_idx

    def t_median(self) -> int:
        return 5

    def generate_final_clauses(self) -> list[NamedClause]:
        clauses: list[NamedClause] = []
        for i in range(Variable.cube_size - 2):
            for j in range(i):
                for idx in Var.Edges.x.idx_range():
                    var_i = Var.Edges.x(
                        self.edge_idx * (Variable.cube_size - 2) + i,
                        idx,
                        Variable.t_max,
                    )
                    var_j = Var.Edges.x(
                        self.edge_idx * (Variable.cube_size - 2) + j,
                        idx,
                        Variable.t_max,
                    )

                    clauses.append(
                        (
                            f"Etat final SameEdges x, edge {self.edge_idx}, i={i}, j={j} clause 1",
                            [var_i, -var_j],
                        )
                    )
                    clauses.append(
                        (
                            f"Etat final SameEdges x, edge {self.edge_idx}, i={i}, j={j} clause 2",
                            [-var_i, var_j],
                        )
                    )

        return clauses

In [ ]:
def find_optimal_3x3x3(self, cube: RubiksCube) -> list[Var.Actions]:
    """
    Trouve la solution optimale pour un Rubik's Cube 3x3x3.
    """
    
    assert cube.size == 3, "Cube size must be 3"
    
    Variable.cube_size = 3

    actions = self.find_optimal_with_steps(
        cube,
        steps=[
            Corners(),
            EdgeOrientation() + Centers(),
            EdgePositionOnCircle(),
            EdgePosition(),
        ],
    )

    return solver.remove_duplicates(actions)

RubiksCubeSolver.find_optimal_3x3x3 = find_optimal_3x3x3

In [ ]:
def convert_to_3x3x3(cube: RubiksCube) -> RubiksCube:
    cube3x3x3 = RubiksCube(3)
    for face in Face:
        for x, y in np.ndindex(3, 3):
            new_x = -1 if x == 2 else x
            new_y = -1 if y == 2 else y
            
            cube3x3x3.faces[face][x, y] = cube.faces[face][new_x, new_y]
    return cube3x3x3

def find_optimal_nxnxn(self, cube: RubiksCube, steps: list[Step]) -> list[Var.Actions]:
    """
    Trouve la solution optimale pour un Rubik's Cube nxnxn.
    """
    cube = cube.copy()
    actions = self.find_optimal_with_steps(cube, steps)
    for action in actions:
        cube.rotate(action.face, action.direction, action.depth)
    
    if cube.size <= 3:
        return actions
    
    cube3x3x3 = convert_to_3x3x3(cube)
    actions3x3x3 = self.find_optimal_3x3x3(cube3x3x3)
    for action in actions3x3x3:
        if action.depth == 0:
            actions.append(action)
            continue

        actions += [
            Var.Actions(action.face, action.direction, depth, action.t)
            for depth in range(1, cube.size - 1)
        ]
    
    return self.remove_duplicates(actions)

RubiksCubeSolver.find_optimal_nxnxn = find_optimal_nxnxn

In [ ]:
Variable.cube_size = 4

rubiks_cube = RubiksCube(4)
rubiks_cube.shuffle(replace_origin=True)

solver = RubiksCubeSolver("rubiks_cube.cnf")

actions = solver.find_optimal_nxnxn(
    rubiks_cube,
    steps=[
        WhiteAndYellowCentersLine(0),
        WhiteAndYellowCentersLine(1),
        CentersNotWhiteOrYellow(),
        SameEdges(0),
        SameEdges(1),
        SameEdges(2),
        SameEdges(3),
        SameEdges(4),
        SameEdges(5),
        SameEdges(6),
        SameEdges(7),
        SameEdges(8),
        SameEdges(9),
        SameEdges(10),
    ],
)

print(f"Solved in {len(actions)} moves")
moves = [
    RubiksCube.move_to_str(action.face, action.direction, action.depth)
    for action in actions
]

rubiks_cube.animate(RubiksCube.parse_moves(moves), speed=5)


AssertionError: Origin not placed correctly: 6